
# [Lux AI] Working Title Bot
The code structure and logic, and version updates are elaborated in the comment section.

I hope this can be a useful template for you to work on your bot on.
You are recommended to edit on a clone/fork of [my repository](https://github.com/tonghuikang/lux-ai-2021) with your favorite IDE.
You can submit the zip the repository to the competition. This notebook is generated with `generate_notebook.py`.

Regardless, do feel free to clone this notebook and submit `submission.tar.gz` under the "Data" tab.


In [ ]:
!pip install kaggle-environments -U > /dev/null
!cp -r ../input/lux-ai-2021/* .

# Agent Logic
The following scipts contain the algorithms that the agent uses.
The algorithm is described in the comments.
Feel free to ask for more clarification.


In [ ]:
%%writefile agent.py
import os
import time
import pickle

import builtins as __builtin__

from lux.game import Game, Missions, Observation

from make_actions import make_city_actions, make_unit_missions, make_unit_actions, make_unit_actions_supplementary
from make_annotations import annotate_game_state, annotate_missions, annotate_movements, filter_cell_annotations
from imitation_agent import get_imitation_action

game_state = Game()
missions = Missions()


def game_logic(game_state: Game, missions: Missions, observation: Observation, DEBUG=False):
    if DEBUG: print = __builtin__.print
    else: print = lambda *args: None

    game_state.compute_start_time = time.time()
    game_state.calculate_features(missions)
    censoring = game_state.is_symmetrical()
    state_annotations = annotate_game_state(game_state)
    reset_missions, actions_by_cities = make_city_actions(game_state, missions, DEBUG=DEBUG)
    if reset_missions or not game_state.player.researched_coal():
        print("reset_missions")
        missions.reset_missions(game_state.player.research_points,
                                game_state.convolve(game_state.coal_exist_matrix),
                                game_state.convolve(game_state.uranium_exist_matrix))
        game_state.calculate_features(missions)
    actions_by_units_initial = make_unit_actions_supplementary(game_state, missions, observation, initial=True, DEBUG=DEBUG)
    cluster_annotations_and_ejections_pre = make_unit_missions(game_state, missions, is_subsequent_plan=False, DEBUG=DEBUG)
    # missions, pre_actions_by_units = make_unit_actions(game_state, missions, DEBUG=DEBUG)
    missions, actions_by_units = make_unit_actions(game_state, missions, DEBUG=DEBUG)
    cluster_annotations_and_ejections = make_unit_missions(game_state, missions, is_subsequent_plan=True, DEBUG=DEBUG)
    actions_by_units_supplementary = make_unit_actions_supplementary(game_state, missions, observation, DEBUG=DEBUG)
    movement_annotations = annotate_movements(game_state, actions_by_units)
    mission_annotations = annotate_missions(game_state, missions, DEBUG=DEBUG)

    print("actions_by_cities", actions_by_cities)
    print("actions_by_units_initial", actions_by_units_initial)
    # print("cluster_annotations_and_ejections_pre", cluster_annotations_and_ejections_pre)
    # print("pre_actions_by_units", pre_actions_by_units)
    print("cluster_annotations_and_ejections", cluster_annotations_and_ejections)
    print("mission_annotations", mission_annotations)
    print("actions_by_units", actions_by_units)
    print("actions_by_units_supplementary", actions_by_units_supplementary)
    print("state_annotations", state_annotations)
    print("movement_annotations", movement_annotations)
    # actions = actions_by_cities + actions_by_units_initial + pre_actions_by_units + actions_by_units + actions_by_units_supplementary
    # actions += cluster_annotations_and_ejections + cluster_annotations_and_ejections_pre
    # actions += mission_annotations + movement_annotations + state_annotations
    actions = actions_by_cities + state_annotations + mission_annotations + actions_by_units_initial
    actions += cluster_annotations_and_ejections_pre + cluster_annotations_and_ejections + actions_by_units + actions_by_units_supplementary + movement_annotations
    actions = filter_cell_annotations(actions, game_state)
    if censoring: actions = []
    return actions, game_state, missions


def agent(observation: Observation, configuration, DEBUG=False):
    if DEBUG: print = __builtin__.print
    else: print = lambda *args: None

    del configuration  # unused
    global game_state, missions

    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state.player_id = observation.player
        game_state._update(observation["updates"][2:])
        game_state.fix_iteration_order()
    else:
        # actually rebuilt and recomputed from scratch
        game_state._update(observation["updates"])

    if not os.environ.get('GFOOTBALL_DATA_DIR', ''):  # on Kaggle compete, do not save items
        str_step = str(observation["step"]).zfill(3)
        with open('snapshots/observation-{}-{}.pkl'.format(str_step, game_state.player_id), 'wb') as handle:
            pickle.dump(observation, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('snapshots/game_state-{}-{}.pkl'.format(str_step, game_state.player_id), 'wb') as handle:
            pickle.dump(game_state, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('snapshots/missions-{}-{}.pkl'.format(str_step, game_state.player_id), 'wb') as handle:
            pickle.dump(missions, handle, protocol=pickle.HIGHEST_PROTOCOL)

    actions, game_state, missions = game_logic(game_state, missions, observation)
    return actions


In [ ]:
%%writefile make_actions.py
# functions executing the actions

import builtins as __builtin__
from typing import Tuple, List, Set

from lux.game import Game, Mission, Missions, Observation, cleanup_missions
from lux.game_objects import Cargo, CityTile, Unit, City
from lux.game_position import Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
import lux.annotate as annotate

from heuristics import find_best_cluster
from imitation_agent import get_imitation_action

DIRECTIONS = Constants.DIRECTIONS


def make_city_actions(game_state: Game, missions: Missions, DEBUG=False) -> List[str]:
    if DEBUG: print = __builtin__.print
    else: print = lambda *args: None

    player = game_state.player
    cleanup_missions(game_state, missions, DEBUG=DEBUG)
    game_state.repopulate_targets(missions)

    units_cap = sum([len(x.citytiles) for x in player.cities.values()])
    units_cap = min(80, units_cap)
    units_cnt = len(player.units)  # current number of units

    actions: List[str] = []
    reset_missions = False

    def do_research(city_tile: CityTile, annotation: str=""):
        nonlocal reset_missions
        action = city_tile.research()
        game_state.player.research_points += 1
        actions.append(action)
        if annotation:
            actions.append(annotate.text(city_tile.pos.x, city_tile.pos.y, annotation))
        city_tile.cooldown += 10

        # reset all missions
        if game_state.player.research_points == 50:
            print("delete missions at 50 rp")
            reset_missions = True
        if game_state.player.research_points == 200:
            print("delete missions at 200 rp")
            reset_missions = True

    def build_worker(city_tile: CityTile, annotation: str=""):
        nonlocal units_cnt

        if tuple(city_tile.pos) in game_state.avoid_building_workers_xy_set:
            return

        action = city_tile.build_worker()
        actions.append(action)
        units_cnt += 1
        game_state.citytiles_with_new_units_xy_set.add(tuple(city_tile.pos))
        if annotation:
            actions.append(annotate.text(city_tile.pos.x, city_tile.pos.y, annotation))
        city_tile.cooldown += 10

        # fake unit and mission to simulate targeting current position
        # if unit limit is not reached
        if units_cnt <= units_cap:
            unit = Unit(game_state.player_id, 0, city_tile.cityid, city_tile.pos.x, city_tile.pos.y,
                        cooldown=6, wood=0, coal=0, uranium=0)  # add dummy unit for targeting purposes
            game_state.players[game_state.player_id].units.append(unit)
            game_state.player.units_by_id[city_tile.cityid] = unit
            mission = Mission(city_tile.cityid, city_tile.pos, details="born", delays=99)
            missions.add(mission)
            game_state.unit_ids_with_missions_assigned_this_turn.add(city_tile.cityid)
            print(missions)

    def build_cart(city_tile: CityTile, annotation: str=""):
        nonlocal units_cnt
        action = city_tile.build_cart()
        actions.append(action)
        units_cnt += 1
        game_state.citytiles_with_new_units_xy_set.add(tuple(city_tile.pos))
        if annotation:
            actions.append(annotate.text(city_tile.pos.x, city_tile.pos.y, annotation))
        city_tile.cooldown += 10

    city_tiles: List[CityTile] = []
    for city in player.cities.values():
        for city_tile in city.citytiles:
            city_tiles.append(city_tile)
    if not city_tiles:
        return False, []


    def calculate_city_cluster_bonus(pos: Position):
        current_leader = game_state.xy_to_resource_group_id.find(tuple(pos))
        units_mining_on_current_cluster = game_state.resource_leader_to_locating_units[current_leader] & game_state.resource_leader_to_targeting_units[current_leader]
        resource_size_of_current_cluster = game_state.xy_to_resource_group_id.get_point(current_leader)
        return resource_size_of_current_cluster / (1+len(units_mining_on_current_cluster))


    city_tiles.sort(key=lambda city_tile:(
        - calculate_city_cluster_bonus(city_tile.pos),
        - max(1, game_state.distance_from_player_units[city_tile.pos.y,city_tile.pos.x])  # max because we assume that it will leave
        + max(0, game_state.distance_from_opponent_assets[city_tile.pos.y,city_tile.pos.x] * 3/2),
        + game_state.player_units_matrix[city_tile.pos.y,city_tile.pos.x],
        - game_state.distance_from_collectable_resource[city_tile.pos.y,city_tile.pos.x],
        - game_state.distance_from_edge[city_tile.pos.y,city_tile.pos.x],
        city_tile.pos.x * game_state.x_order_coefficient,
        city_tile.pos.y * game_state.y_order_coefficient
    ),)

    print("".join([str((city_tile.pos.x, city_tile.pos.y,
        - calculate_city_cluster_bonus(city_tile.pos),
        - max(1, game_state.distance_from_player_units[city_tile.pos.y,city_tile.pos.x])  # max because we assume that it will leave
        + max(0, game_state.distance_from_opponent_assets[city_tile.pos.y,city_tile.pos.x] / 2)
        + game_state.player_units_matrix[city_tile.pos.y,city_tile.pos.x],
        - game_state.distance_from_collectable_resource[city_tile.pos.y,city_tile.pos.x],
        - game_state.distance_from_edge[city_tile.pos.y,city_tile.pos.x],
        city_tile.pos.x * game_state.x_order_coefficient,
        city_tile.pos.y * game_state.y_order_coefficient
    ),) + "\n" for city_tile in city_tiles]))

    for city_tile in city_tiles:
        if not city_tile.can_act():
            continue

        print("city_tile values", -calculate_city_cluster_bonus(city_tile.pos),
        - max(1, game_state.distance_from_player_units[city_tile.pos.y,city_tile.pos.x])  # max because we assume that it will leave
        + game_state.distance_from_opponent_assets[city_tile.pos.y,city_tile.pos.x],
        - game_state.distance_from_collectable_resource[city_tile.pos.y,city_tile.pos.x],
        - game_state.distance_from_edge[city_tile.pos.y,city_tile.pos.x],
        city_tile.pos.x * game_state.x_order_coefficient,
        city_tile.pos.y * game_state.y_order_coefficient)

        unit_limit_exceeded = (units_cnt >= units_cap)

        if player.researched_uranium() and unit_limit_exceeded:
            # you cannot build units because you have reached your limits
            print("limit reached", city_tile.cityid, tuple(city_tile.pos))
            continue

        nearest_resource_distance = game_state.distance_from_collectable_resource[city_tile.pos.y, city_tile.pos.x]
        travel_range_emptyhanded = 1 + game_state.turns_to_night // GAME_CONSTANTS["PARAMETERS"]["UNIT_ACTION_COOLDOWN"]["WORKER"]
        resource_in_travel_range = nearest_resource_distance <= travel_range_emptyhanded
        if game_state.player.researched_uranium():
            resource_in_travel_range = True

        cluster_leader = game_state.xy_to_resource_group_id.find(tuple(city_tile.pos))
        cluster_unit_limit_exceeded = \
            game_state.xy_to_resource_group_id.get_point(tuple(city_tile.pos)) <= len(game_state.resource_leader_to_locating_units[cluster_leader])

        # standard process of building workers
        if resource_in_travel_range and not unit_limit_exceeded and not cluster_unit_limit_exceeded:
            print("build_worker WA", city_tile.cityid, city_tile.pos.x, city_tile.pos.y, nearest_resource_distance, travel_range_emptyhanded)
            build_worker(city_tile, "WA")
            continue

        # allow cities to build workers even if cluster_unit_limit_exceeded
        # because uranium is researched or scouting for advanced resources
        # require resource_in_travel_range
        if player.researched_uranium() or (units_cnt <= units_cap//4):
            if resource_in_travel_range:
                # but do not build workers beside wood to conserve wood
                if game_state.wood_side_matrix[city_tile.pos.y, city_tile.pos.x] == 0:
                    print("supply workers WB", city_tile.cityid, city_tile.pos.x, city_tile.pos.y, nearest_resource_distance, travel_range_emptyhanded)
                    build_worker(city_tile, "WB")
                    continue

        # build worker and move to adjacent if there are no workers nearby
        if nearest_resource_distance == 2 and game_state.distance_from_player_units[city_tile.pos.y, city_tile.pos.x] > 2:
            print("supply workers WC", city_tile.cityid, city_tile.pos.x, city_tile.pos.y, nearest_resource_distance, travel_range_emptyhanded)
            build_worker(city_tile, "WC")
            continue


        if not player.researched_uranium():
            # give up researching to allow building of units at turn 359
            if game_state.turn < 10:
                actions.append(annotate.text(city_tile.pos.x, city_tile.pos.y, "NS"))
            elif game_state.turn < 350:
                print("research RA", tuple(city_tile.pos))
                do_research(city_tile, "RA")
                continue
            else:
                actions.append(annotate.text(city_tile.pos.x, city_tile.pos.y, "NE"))

        # extend carts to fetch resource
        if 10 < game_state.player.cities[city_tile.cityid].night_fuel_duration < 30 and game_state.is_day_time:
            if game_state.player.cities[city_tile.cityid].citytiles.__len__() > 5:
                if not unit_limit_exceeded:
                    build_cart(city_tile, "NC")

        # easter egg - build carts or research for fun when there is no resource left
        if game_state.map_resource_count == 0 and game_state.is_day_time:
            if not unit_limit_exceeded:
                if game_state.player.cities[city_tile.cityid].fuel_needed_for_game < 0:
                    print("research NC", tuple(city_tile.pos))
                    do_research(city_tile, "RA")
                else:
                    build_cart(city_tile, "NC")

        # build workers at end of game
        if game_state.turn == 359:
            print("build_worker WE", city_tile.cityid, city_tile.pos.x, city_tile.pos.y, nearest_resource_distance, travel_range_emptyhanded)
            build_cart(city_tile, "WE")
            continue

        # otherwise don't do anything

    return reset_missions, actions


def make_unit_missions(game_state: Game, missions: Missions, is_subsequent_plan=False, DEBUG=False) -> Missions:
    if DEBUG: print = __builtin__.print
    else: print = lambda *args: None

    player = game_state.player
    cleanup_missions(game_state, missions, DEBUG=DEBUG)
    actions_ejections = []

    cluster_annotations = []

    # attempt to eject coal/uranium, unit is the one ejecting
    for unit in player.units:
        if not is_subsequent_plan:
            continue
        # unit is the one ejecting
        unit: Unit = unit
        if not unit.can_act():
            continue

        # source unit has plenty of uranium
        if not (unit.cargo.uranium >= 30):
            continue

        # source unit not in empty tile
        # if tuple(unit.pos) not in game_state.convolved_collectable_tiles_xy_set:
        #     continue

        for adj_unit in player.units:
            # adj_unit is the one being ejected
            adj_unit: Unit = adj_unit
            if not adj_unit.can_act():
                continue

            # source unit is not the target unit
            if adj_unit.id == unit.id:
                continue

            # source unit is beside target unit
            if adj_unit.pos - unit.pos != 1:
                continue

            # adjacent unit is in city tile
            if tuple(adj_unit.pos) not in game_state.player_city_tile_xy_set:
                continue

            # adjacent unit is beside an empty tile
            if game_state.distance_from_empty_tile[adj_unit.pos.y, adj_unit.pos.x] != 1:
                continue

            # adjacent unit is inside a city that can survive the night
            if game_state.matrix_player_cities_nights_of_fuel_required_for_night[adj_unit.pos.y, adj_unit.pos.x] >= 0:
                continue

            # execute actions for ejection
            action_1 = unit.transfer(adj_unit.id, unit.cargo.get_most_common_resource(), 100)
            for direction,(dx,dy) in zip(game_state.dirs, game_state.dirs_dxdy[:-1]):
                xx,yy = adj_unit.pos.x + dx, adj_unit.pos.y + dy
                if (xx,yy) in game_state.empty_tile_xy_set:
                    print("ejecting", unit.id, unit.pos, adj_unit.id, adj_unit.pos, "->")
                    action_2 = adj_unit.move(direction)
                    actions_ejections.append(action_1)
                    actions_ejections.append(action_2)
                    actions_ejections.append(annotate.text(unit.pos.x, unit.pos.y, "🟡", 50))
                    unit.cargo = Cargo()
                    adj_unit.cargo.wood += 100  # not correct, but simulated
                    unit.cooldown += 2
                    adj_unit.cooldown += 2
                    game_state.player_units_matrix[adj_unit.pos.y,adj_unit.pos.x] -= 1
                    break
            else:
                break

           # add missions for ejection
            print("plan mission ejection success", xx, yy)

            # if successful
            if unit.id in missions:
                print("delete mission because ejecting", unit.id, unit.pos)
                del missions[unit.id]
            if adj_unit.id in missions:
                print("delete mission because ejected", adj_unit.id, adj_unit.pos)
                del missions[adj_unit.id]
            game_state.unit_ids_with_missions_assigned_this_turn.add(adj_unit.id)
            game_state.ejected_units_set.add(adj_unit.id)

            # break loop since partner for unit is found
            if not unit.can_act():
                break


    # attempt to eject, unit is the one ejecting
    for unit in player.units:
        continue
        # unit is the one ejecting
        unit: Unit = unit
        if not unit.can_act():
            continue

        if is_subsequent_plan and game_state.distance_from_opponent_assets[unit.pos.y, unit.pos.x] < 5:
            continue

        # source unit not in empty tile
        if tuple(unit.pos) in game_state.buildable_tile_xy_set:
            continue

        # source unit has almost full resources
        if unit.get_cargo_space_used() < 96 and unit.cargo.get_most_common_resource_count() < 40:
            continue

        print("considering unit", unit.id)

        for adj_unit in player.units:
            # adj_unit is the one being ejected
            adj_unit: Unit = adj_unit
            if not adj_unit.can_act():
                continue

            # source unit is not the target unit
            if adj_unit.id == unit.id:
                continue

            # source unit is not beside target unit
            if adj_unit.pos - unit.pos != 1:
                continue

            # adjacent unit is in city tile
            if tuple(adj_unit.pos) not in game_state.player_city_tile_xy_set:
                continue

            # adjacent unit is beside an empty tile
            if game_state.distance_from_empty_tile[adj_unit.pos.y, adj_unit.pos.x] != 1:
                continue

            # temporarily augment night travel range
            adj_unit.cargo.wood += unit.cargo.get_most_common_resource_count()
            adj_unit.compute_travel_range((game_state.turns_to_night, game_state.turns_to_dawn, game_state.is_day_time),)
            best_position, best_cell_value, cluster_annotation = find_best_cluster(game_state, adj_unit, DEBUG=DEBUG, explore=True, ref_pos=unit.pos)
            distance_of_best = game_state.retrieve_distance(adj_unit.pos.x, adj_unit.pos.y, best_position.x, best_position.y)
            adj_unit.cargo.wood -= unit.cargo.get_most_common_resource_count()
            adj_unit.compute_travel_range((game_state.turns_to_night, game_state.turns_to_dawn, game_state.is_day_time),)

            print("eligible mission ejection", unit.id, unit.pos, best_cell_value)

            # no suitable candidate found
            if best_cell_value == [0,0,0,0]:
                continue

            # do not eject and return to the same cluster
            if game_state.xy_to_resource_group_id.find(tuple(best_position)) == game_state.xy_to_resource_group_id.find(tuple(unit.pos)):
                continue

            # add missions for ejection
            print("plan mission ejection", adj_unit.id, adj_unit.pos, "->", best_position, best_cell_value)

            # execute actions for ejection
            action_1 = unit.transfer(adj_unit.id, unit.cargo.get_most_common_resource(), 100)
            for direction,(dx,dy) in zip(game_state.dirs, game_state.dirs_dxdy[:-1]):
                xx,yy = adj_unit.pos.x + dx, adj_unit.pos.y + dy
                if (xx,yy) in game_state.empty_tile_xy_set:
                    if game_state.retrieve_distance(xx, yy, best_position.x, best_position.y) > distance_of_best:
                        continue
                    if Position(xx,yy) - best_position > unit.pos - best_position:
                        continue
                    if Position(xx,yy) - best_position > adj_unit.pos - best_position:
                        continue
                    print("ejecting", unit.id, unit.pos, adj_unit.id, adj_unit.pos, direction, "->", best_position)
                    game_state.occupied_xy_set.add((xx,yy),)
                    game_state.empty_tile_xy_set.remove((xx,yy))
                    action_2 = adj_unit.move(direction)
                    actions_ejections.append(action_1)
                    actions_ejections.append(action_2)
                    actions_ejections.append(annotate.text(unit.pos.x, unit.pos.y, "🔴", 50))
                    unit.cargo = Cargo()
                    adj_unit.cargo.wood += 100  # not correct, but simulated
                    unit.cooldown += 2
                    adj_unit.cooldown += 2
                    game_state.player_units_matrix[adj_unit.pos.y,adj_unit.pos.x] -= 1
                    break
            else:
                break

           # add missions for ejection
            print("plan mission ejection success", xx, yy)

            # if successful
            if unit.id in missions:
                print("delete mission because ejecting", unit.id, unit.pos)
                del missions[unit.id]
            if adj_unit.id in missions:
                print("delete mission because ejected", adj_unit.id, adj_unit.pos)
                del missions[adj_unit.id]
            mission = Mission(adj_unit.id, best_position, delays=distance_of_best)
            missions.add(mission)
            game_state.unit_ids_with_missions_assigned_this_turn.add(adj_unit.id)
            game_state.ejected_units_set.add(adj_unit.id)
            cluster_annotations.extend(cluster_annotation)

            # break loop since partner for unit is found
            if not unit.can_act():
                break

    # main sequence
    for unit in player.units:
        if unit.id in game_state.unit_ids_with_missions_assigned_this_turn:
            continue
        # mission is planned regardless whether the unit can act
        current_mission: Mission = missions[unit.id] if unit.id in missions else None
        current_target = current_mission.target_position if current_mission else None

        # avoid sharing the same target
        game_state.repopulate_targets(missions)

        # do not make missions from a fortress
        if game_state.distance_from_floodfill_by_player_city[unit.pos.y, unit.pos.x] > 1:
            # if you are carrying some wood
            if unit.cargo.wood >= 40:
                # assuming uranium has yet to be researched
                if not game_state.player.researched_uranium():
                    # allow building beside sustainable city
                    if not game_state.preferred_buildable_tile_matrix[unit.pos.y,unit.pos.x]:
                        print("no mission from fortress", unit.id)
                        continue

        # do not make missions if you could mine uranium from a citytile that is not fueled for the night
        if game_state.matrix_player_cities_nights_of_fuel_required_for_night[unit.pos.y, unit.pos.x] > 0 or (
            game_state.distance_from_opponent_assets[unit.pos.y, unit.pos.x] <= 3 and
            game_state.matrix_player_cities_nights_of_fuel_required_for_game[unit.pos.y, unit.pos.x] > 0):
            if game_state.player.researched_uranium():
                if game_state.convolved_uranium_exist_matrix[unit.pos.y, unit.pos.x] > 0:
                    if tuple(unit.pos) not in game_state.citytiles_with_new_units_xy_set:
                        if game_state.player_units_matrix[unit.pos.y, unit.pos.x] == 1:
                            print("stay and mine uranium", unit.id, unit.pos)
                            # unless the citytile is producing new units
                            continue

        # do not make missions if you could mine coal from a citytile that is not fueled for the night
        if game_state.matrix_player_cities_nights_of_fuel_required_for_night[unit.pos.y, unit.pos.x] > 0:
            if game_state.player.researched_coal():
                if game_state.convolved_coal_exist_matrix[unit.pos.y, unit.pos.x] > 0:
                    if tuple(unit.pos) not in game_state.citytiles_with_new_units_xy_set:
                        if game_state.player_units_matrix[unit.pos.y, unit.pos.x] == 1:
                            print("stay and mine coal", unit.id, unit.pos)
                            # unless the citytile is producing new units
                            continue

        current_leader = game_state.xy_to_resource_group_id.find(tuple(unit.pos))
        units_mining_on_current_cluster = game_state.resource_leader_to_locating_units[current_leader] & game_state.resource_leader_to_targeting_units[current_leader]
        resource_size_of_current_cluster = game_state.xy_to_resource_group_id.get_point(current_leader)
        current_cluster_load = len(units_mining_on_current_cluster) / (0.01+resource_size_of_current_cluster)

        # if you are targeting your own cluster you are at and you have at least 60 wood and close to edge
        targeting_current_cluster = unit.id not in missions or (unit.id in missions and \
                                    game_state.xy_to_resource_group_id.find(tuple(unit.pos)) == \
                                    game_state.xy_to_resource_group_id.find(tuple(missions.get_target_of_unit(unit.id))))
        full_resources_on_next_turn = not ((unit.get_cargo_space_used() + game_state.resource_collection_rate[unit.pos.y, unit.pos.x] * (1 + int(unit.cooldown)) < 100
                                           ) or (31 < game_state.turn%40 <= 37))

        print("housing test", unit.id, unit.pos, unit.id in missions, targeting_current_cluster, full_resources_on_next_turn)

        # if far away from enemy units, attempt to send units to empty cluster
        if game_state.distance_from_opponent_assets[unit.pos.y, unit.pos.x] > 10:
            if not unit.can_act():
                pass
            elif not full_resources_on_next_turn:
                best_position, best_cell_value, cluster_annotation = find_best_cluster(game_state, unit, DEBUG=DEBUG, require_empty_target=True)
                distance_from_best_position = game_state.retrieve_distance(unit.pos.x, unit.pos.y, best_position.x, best_position.y)
                if best_cell_value > [0,0,0,0]:
                    print("force empty cluster", unit.id, unit.pos, "->", best_position, best_cell_value)
                    mission = Mission(unit.id, best_position, delays=distance_from_best_position)
                    missions.add(mission)
                    game_state.unit_ids_with_missions_assigned_this_turn.add(unit.id)
                    cluster_annotations.extend(cluster_annotation)
                    continue


        # you consider building a citytile only if you are currently targeting the cluster you are in
        if targeting_current_cluster and False:

            def get_best_eligible_tile(xy_set: Set) -> Tuple[Position, int]:

                best_heuristic = -999
                nearest_position: Position = unit.pos
                for dx,dy in game_state.dirs_dxdy[:-1]:
                    xx,yy = unit.pos.x+dx, unit.pos.y+dy
                    if (xx,yy) in xy_set:
                        if (xx,yy) in game_state.player_units_xy_set and (xx,yy) != tuple(unit.pos):
                            continue
                        if (xx,yy) in game_state.targeted_for_building_xy_set:
                            # we allow units to build at a tile that is targeted but not for building
                            if not current_target:
                                # definitely you are not the one targeting it
                                continue
                            if current_target and (xx,yy) != tuple(current_target):
                                continue
                        if unit.get_cargo_space_used() + 2*game_state.resource_collection_rate[yy, xx] >= 100:
                            heuristic = - game_state.distance_from_opponent_assets[yy,xx] - game_state.distance_from_resource_median[yy,xx]
                            if heuristic > best_heuristic:
                                best_heuristic = heuristic
                                nearest_position = Position(xx,yy)
                if best_heuristic > -999:
                    return True, nearest_position
                else:
                    return False, None


            relocation_to_preferred = (game_state.distance_from_preferred_buildable[unit.pos.y, unit.pos.x] <= 1 and
                                       unit.get_cargo_space_used() == 100 and 0 < game_state.turn%40 < 28 and
                                       game_state.distance_from_opponent_assets[unit.pos.y, unit.pos.x] > 2
                                       ) or (
                                       game_state.distance_from_preferred_buildable[unit.pos.y, unit.pos.x] == 0 and
                                       unit.get_cargo_space_used() == 100 and 0 < game_state.turn%40 <= 31
                                       )

            # if you can move one step to a building that can survive a night, build there
            if relocation_to_preferred:
                has_found, new_pos = get_best_eligible_tile(game_state.preferred_buildable_tile_xy_set)
                if tuple(unit.pos) in game_state.preferred_buildable_tile_xy_set:
                    has_found, new_pos = True, unit.pos
                if has_found:
                    print("relocation_to_preferred", unit.id, unit.pos, "->", new_pos)
                    mission = Mission(unit.id, new_pos, unit.build_city(), delays=2)
                    missions.add(mission)
                    game_state.unit_ids_with_missions_assigned_this_turn.add(unit.id)
                    annotation = annotate.text(unit.pos.x, unit.pos.y, "R1")
                    cluster_annotations.append(annotation)
                    continue

            relocation_to_probable =  (game_state.distance_from_probably_buildable[unit.pos.y, unit.pos.x] <= 1 and
                                       unit.get_cargo_space_used() == 100 and 0 < game_state.turn%40 < 28 and
                                       game_state.distance_from_opponent_assets[unit.pos.y, unit.pos.x] > 3 and
                                       game_state.turn > 40 and current_cluster_load > 1/2
                                       ) or (
                                       game_state.distance_from_probably_buildable[unit.pos.y, unit.pos.x] == 0 and
                                       unit.get_cargo_space_used() == 100 and 0 < game_state.turn%40 <= 30
                                       )

            # if the cluster is crowded, consider building at a corner (which is not directly collecting resources)
            if relocation_to_probable:
                has_found, new_pos = get_best_eligible_tile(game_state.probably_buildable_tile_xy_set)
                if tuple(unit.pos) in game_state.probably_buildable_tile_xy_set:
                    has_found, new_pos = True, unit.pos
                if has_found:
                    print("relocation_to_probable", unit.id, unit.pos, "->", new_pos)
                    mission = Mission(unit.id, new_pos, unit.build_city(), delays=2)
                    missions.add(mission)
                    game_state.unit_ids_with_missions_assigned_this_turn.add(unit.id)
                    annotation = annotate.text(unit.pos.x, unit.pos.y, "R2")
                    cluster_annotations.append(annotation)
                    continue

            # if you will have full resources on the next turn and on buildable tile, stay and build
            if full_resources_on_next_turn and tuple(unit.pos) in game_state.buildable_tile_xy_set:
                if game_state.distance_from_player_citytiles[unit.pos.y, unit.pos.x] == 1 or \
                    game_state.distance_from_collectable_resource[unit.pos.y, unit.pos.x] == 1:
                    print("stay on location", unit.id, unit.pos)
                    mission = Mission(unit.id, unit.pos, unit.build_city(), delays=2)
                    missions.add(mission)
                    game_state.unit_ids_with_missions_assigned_this_turn.add(unit.id)
                    annotation = annotate.text(unit.pos.x, unit.pos.y, "R3")
                    cluster_annotations.append(annotation)
                    continue

            if not full_resources_on_next_turn:
                has_found, new_pos = get_best_eligible_tile(game_state.buildable_and_convolved_collectable_tile_xy_set)
                if has_found:
                    print("relocation to better one", unit.id, unit.pos, "->", new_pos)
                    mission = Mission(unit.id, new_pos, unit.build_city(), delays=2)
                    missions.add(mission)
                    game_state.unit_ids_with_missions_assigned_this_turn.add(unit.id)
                    annotation = annotate.text(unit.pos.x, unit.pos.y, "R4")
                    cluster_annotations.append(annotation)
                    continue

            if full_resources_on_next_turn:
                has_found, new_pos = get_best_eligible_tile(game_state.buildable_and_convolved_collectable_tile_xy_set)
                if has_found:
                    print("build now", unit.id, unit.pos, "->", new_pos)
                    mission = Mission(unit.id, new_pos, unit.build_city(), delays=2)
                    missions.add(mission)
                    game_state.unit_ids_with_missions_assigned_this_turn.add(unit.id)
                    annotation = annotate.text(unit.pos.x, unit.pos.y, "R5")
                    cluster_annotations.append(annotation)
                    continue

        if unit.id in missions:
            mission: Mission = missions[unit.id]
            if mission.target_position == unit.pos:
                # take action and not make missions if already at position
                continue

        if unit.id in missions:
            # the mission will be recaluated if the unit fails to make a move after make_unit_actions
            continue

        # preemptive homing mission
        if tuple(unit.pos) not in game_state.convolved_collectable_tiles_xy_set:
          if unit.fuel_potential > 230:
            # if there is a citytile nearby already
            print("consider homing two", unit.id)
            homing_distance, homing_position = game_state.find_nearest_city_requiring_fuel(
                unit, require_reachable=True, enforce_night=False,
                minimum_size=3, maximum_distance=(unit.cargo.uranium + unit.cargo.coal)//3, DEBUG=DEBUG)
            if unit.pos != homing_position:
                print("homing two", unit.id, unit.pos, homing_position)
                mission = Mission(unit.id, homing_position, details="homing two", delays=homing_distance + 2)
                missions.add(mission)
                game_state.unit_ids_with_missions_assigned_this_turn.add(unit.id)
                annotation = annotate.text(unit.pos.x, unit.pos.y, "H2")
                cluster_annotations.append(annotation)
                continue

        if tuple(unit.pos) not in game_state.convolved_collectable_tiles_xy_set or game_state.distance_from_opponent_assets[unit.pos.y, unit.pos.x] > 2:
          if unit.cargo.uranium > 0 and unit.cargo.get_most_common_resource() == "uranium" and False:
            # if there is a citytile nearby already
            homing_distance, homing_position = game_state.find_nearest_city_requiring_fuel(
                unit, require_reachable=True, require_night=True, enforce_night=True,
                minimum_size=3, maximum_distance=unit.cargo.uranium//3, DEBUG=DEBUG)
            if unit.pos != homing_position:
                print("homing one", unit.id, unit.pos, homing_position, homing_distance)
                mission = Mission(unit.id, homing_position, details="homing", delays=homing_distance + 2)
                missions.add(mission)
                game_state.unit_ids_with_missions_assigned_this_turn.add(unit.id)
                annotation = annotate.text(unit.pos.x, unit.pos.y, "H1")
                cluster_annotations.append(annotation)
                continue

        best_position, best_cell_value, cluster_annotation = find_best_cluster(game_state, unit, DEBUG=DEBUG)
        print(unit.id, best_position, best_cell_value)
        distance_from_best_position = game_state.retrieve_distance(unit.pos.x, unit.pos.y, best_position.x, best_position.y)
        if best_cell_value > [0,0,0,0]:
            print("plan mission adaptative", unit.id, unit.pos, "->", best_position, best_cell_value)
            mission = Mission(unit.id, best_position, delays=distance_from_best_position)
            missions.add(mission)
            game_state.unit_ids_with_missions_assigned_this_turn.add(unit.id)
            cluster_annotations.extend(cluster_annotation)
            continue

        # homing mission
        if unit.get_cargo_space_used() > 0:
            homing_distance, homing_position = game_state.find_nearest_city_requiring_fuel(unit, DEBUG=DEBUG)
            print("homing mission", unit.id, unit.pos, "->", homing_position, homing_distance)
            mission = Mission(unit.id, homing_position, "", details="homing", delays=homing_distance + 2)
            missions.add(mission)
            game_state.unit_ids_with_missions_assigned_this_turn.add(unit.id)
            continue

    return actions_ejections + cluster_annotations


def make_unit_actions(game_state: Game, missions: Missions, DEBUG=False) -> Tuple[Missions, List[str]]:
    if DEBUG: print = __builtin__.print
    else: print = lambda *args: None

    player, opponent = game_state.player, game_state.opponent
    actions = []

    units_with_mission_but_no_action = set(missions.keys())
    prev_actions_len = -1

    # repeat attempting movements for the units until no additional movements can be added
    while prev_actions_len < len(actions):
      prev_actions_len = len(actions)

      for unit in player.units:
        if not unit.use_rule_base:
            continue
        if not unit.can_act():
            units_with_mission_but_no_action.discard(unit.id)
            continue

        # if there is no mission, continue
        if unit.id not in missions:
            units_with_mission_but_no_action.discard(unit.id)
            continue

        mission: Mission = missions[unit.id]
        print("attempting action for", unit.id, unit.pos, "->", mission.target_position)

        # if the location is reached, take action
        if unit.pos == mission.target_position:
            units_with_mission_but_no_action.discard(unit.id)
            print("location reached and make action", unit.id, unit.pos)
            action = mission.target_action

            # do not build city at last light
            if action and action[:5] == "bcity" and 30 <= game_state.turn%40 <= 31:
                if game_state.fuel_collection_rate[unit.pos.y, unit.pos.x] < 23:
                    print("do not build city at last light", unit.id)
                    actions.append(annotate.text(unit.pos.x, unit.pos.y, "NB"))
                    del missions[unit.id]
                    continue

            if action:
                actions.append(action)
                unit.cooldown += 2
            print("mission complete and deleted", unit.id, unit.pos)
            del missions[unit.id]
            continue

        # attempt to move the unit
        direction, pos = attempt_direction_to(game_state, unit, mission.target_position,
                                         avoid_opponent_units=("homing" in mission.details),
                                         use_exact=("homing" in mission.details),
                                         DEBUG=DEBUG)
        if direction == "c":
            continue

        # if carrying full wood, and next location has abundant wood, if on buildable, build house now
        if game_state.convolved_wood_exist_matrix[pos.y, pos.x] > 1:
            if unit.cargo.wood == 100:
                if unit.can_build(game_state.map):
                    actions.append(unit.build_city())
                    unit.cooldown += 2
                    continue

        if True:
            units_with_mission_but_no_action.discard(unit.id)
            action = unit.move(direction)
            print("make move", unit.id, unit.pos, direction, unit.pos.translate(direction, 1))
            game_state.player_units_matrix[unit.pos.y,unit.pos.x] -= 1
            actions.append(action)
            continue

    # if the unit is not able to make an action over two turns, delete the mission
    for unit in game_state.player.units:
        if unit.id not in missions:
            continue
        mission: Mission = missions[unit.id]
        if mission.delays <= 0:
            print("delete mission delay timer over", unit.id, unit.pos, "->", mission.target_position)
            del missions[unit.id]
        elif mission.delays < 2 * (unit.pos - mission.target_position):
            print("delete mission cannot reach in time", unit.id, unit.pos, "->", mission.target_position)
            del missions[unit.id]

    return missions, actions


def make_unit_actions_supplementary(game_state: Game, missions: Missions, observation: Observation,
                                    initial=False, DEBUG=False) -> Tuple[Missions, List[str]]:
    if DEBUG: print = __builtin__.print
    else: print = lambda *args: None

    player, opponent = game_state.player, game_state.opponent
    actions = []

    if initial:
      for unit in player.units:
        if unit.can_act():
            if not unit.use_rule_base:
                actions_from_imitation = get_imitation_action(observation, game_state, unit, DEBUG=DEBUG,
                                                              use_probabilistic_sort=False)
                actions.extend(actions_from_imitation)

    print("units without actions", [unit.id for unit in player.units if unit.can_act()])

    # probably should reduce code repetition in the following lines
    def make_random_move_to_void(unit: Unit, annotation: str = ""):
        if not unit.can_act():
            return
        (xxx,yyy) = (-1,-1)

        # in increasing order of priority

        # attempt to move
        for direction,(dx,dy) in zip(game_state.dirs, game_state.dirs_dxdy[:-1]):
            xx,yy = unit.pos.x + dx, unit.pos.y + dy
            if (xx,yy) not in game_state.occupied_xy_set:
                xxx,yyy = xx,yy
                break

        # attempt to move away from your assets
        for direction,(dx,dy) in zip(game_state.dirs, game_state.dirs_dxdy[:-1]):
            xx,yy = unit.pos.x + dx, unit.pos.y + dy
            if (xx,yy) not in game_state.occupied_xy_set:
                if game_state.distance_from_player_assets[yy,xx] > game_state.distance_from_player_assets[unit.pos.y,unit.pos.x]:
                    xxx,yyy = xx,yy
                    break

        # attempt to move toward enemy assets
        for direction,(dx,dy) in zip(game_state.dirs, game_state.dirs_dxdy[:-1]):
            xx,yy = unit.pos.x + dx, unit.pos.y + dy
            if (xx,yy) not in game_state.occupied_xy_set and (xx,yy) not in game_state.player_city_tile_xy_set:
                if game_state.distance_from_collectable_resource[yy,xx] < game_state.distance_from_collectable_resource[unit.pos.y,unit.pos.x]:
                    xxx,yyy = xx,yy
                    break

        # cart pave roads
        if unit.is_cart():
            for direction,(dx,dy) in zip(game_state.dirs, game_state.dirs_dxdy[:-1]):
                xx,yy = unit.pos.x + dx, unit.pos.y + dy
                if (xx,yy) not in game_state.occupied_xy_set:
                    if game_state.road_level_matrix[yy,xx] < game_state.road_level_matrix[unit.pos.y,unit.pos.x]:
                        xxx,yyy = xx,yy
                        break

        if (xxx,yyy) == (-1,-1):
            return

        xx,yy = xxx,yyy

        if (xx,yy) not in game_state.occupied_xy_set:
            if (xx,yy) not in game_state.player_city_tile_xy_set:
                game_state.occupied_xy_set.add((xx,yy))
            print("make_random_move_to_void", unit.id, unit.pos)
            action = unit.move(direction)
            actions.append(action)
            if annotation:
                actions.append(annotate.text(unit.pos.x, unit.pos.y, annotation))
            unit.cooldown += 2
            game_state.player_units_matrix[unit.pos.y,unit.pos.x] -= 1


    def make_random_move_to_center(unit: Unit, annotation: str = ""):
        if not unit.can_act():
            return
        for direction,(dx,dy) in zip(game_state.dirs, game_state.dirs_dxdy[:-1]):
            xx,yy = unit.pos.x + dx, unit.pos.y + dy
            if (xx,yy) in game_state.player_city_tile_xy_set:
                continue
            if (xx,yy) not in game_state.occupied_xy_set:
                if game_state.distance_from_preferred_median[yy,xx] < game_state.distance_from_preferred_median[unit.pos.y,unit.pos.x]:
                    # attempt to collide together and build additional citytile
                    break
        else:
            return

        if (xx,yy) not in game_state.occupied_xy_set:
            if (xx,yy) not in game_state.player_city_tile_xy_set:
                game_state.occupied_xy_set.add((xx,yy))
            print("make_random_move_to_center", unit.id, unit.pos, direction)
            action = unit.move(direction)
            actions.append(action)
            if annotation:
                actions.append(annotate.text(unit.pos.x, unit.pos.y, annotation))
            unit.cooldown += 2
            game_state.player_units_matrix[unit.pos.y,unit.pos.x] -= 1


    # probably should reduce code repetition in the following lines
    def make_random_move_to_collectable(unit: Unit, annotation: str = ""):
        if not unit.can_act():
            return
        for direction,(dx,dy) in zip(game_state.dirs, game_state.dirs_dxdy[:-1]):
            xx,yy = unit.pos.x + dx, unit.pos.y + dy
            if (xx,yy) not in game_state.occupied_xy_set:
                if (xx,yy) in game_state.convolved_collectable_tiles_xy_set:
                    # attempt to move away from your assets
                    break
        else:
            return

        if (xx,yy) not in game_state.occupied_xy_set:
            if (xx,yy) not in game_state.player_city_tile_xy_set:
                game_state.occupied_xy_set.add((xx,yy))
            print("make_random_move_to_collectable", unit.id, unit.pos)
            action = unit.move(direction)
            actions.append(action)
            if annotation:
                actions.append(annotate.text(unit.pos.x, unit.pos.y, annotation))
            unit.cooldown += 2
            game_state.player_units_matrix[unit.pos.y,unit.pos.x] -= 1


    def make_random_move_to_city(unit: Unit, annotation: str = ""):
        if not unit.can_act():
            return
        for direction,(dx,dy) in zip(game_state.dirs, game_state.dirs_dxdy[:-1]):
            xx,yy = unit.pos.x + dx, unit.pos.y + dy
            if (xx,yy) in game_state.player_city_tile_xy_set:
                if game_state.player_units_matrix[yy,xx] < 1:
                    if (xx,yy) in game_state.convolved_collectable_tiles_xy_set:
                        break
        else:
            return

        if (xx,yy) not in game_state.occupied_xy_set:
            if (xx,yy) not in game_state.player_city_tile_xy_set:
                game_state.occupied_xy_set.add((xx,yy))
            print("make_random_move_to_city", unit.id, unit.pos)
            action = unit.move(direction)
            actions.append(action)
            if annotation:
                actions.append(annotate.text(unit.pos.x, unit.pos.y, annotation))
            unit.cooldown += 2
            game_state.player_units_matrix[unit.pos.y,unit.pos.x] -= 1
            game_state.player_units_matrix[yy,xx] += 1


    def make_random_move_to_city_sustain(unit: Unit, annotation: str = ""):
        if not unit.can_act():
            return
        for direction,(dx,dy) in zip(game_state.dirs, game_state.dirs_dxdy[:-1]):
            xx,yy = unit.pos.x + dx, unit.pos.y + dy
            if (xx,yy) not in game_state.player_city_tile_xy_set:
                continue
            if (xx,yy) in game_state.xy_out_of_map:
                continue
            citytile = game_state.map.get_cell(xx,yy).citytile
            city = game_state.player.cities[citytile.cityid]
            if city.fuel_needed_for_night > 0 and unit.fuel_potential >= city.fuel_needed_for_night:
                print("sustain", unit.id, unit.pos, "->", xx, yy)
                action = unit.move(direction)
                actions.append(action)
                if annotation:
                    actions.append(annotate.text(unit.pos.x, unit.pos.y, annotation))
                unit.cooldown += 2
                game_state.player_units_matrix[unit.pos.y,unit.pos.x] -= 1


    def make_random_transfer(unit: Unit, annotation: str = "", limit_target = False, allowed_target_xy: set = set()):
        if not unit.can_act():
            return
        if unit.get_cargo_space_used() == 0:
            # nothing to transfer
            return
        for direction,(dx,dy) in zip(game_state.dirs, game_state.dirs_dxdy[:-1]):
            xx,yy = unit.pos.x + dx, unit.pos.y + dy
            if (xx,yy) in game_state.xy_out_of_map:
                continue
            if limit_target and (xx,yy) not in allowed_target_xy:
                continue
            adj_unit = game_state.map.get_cell(xx,yy).unit
            if not adj_unit:
                continue
            if adj_unit.id not in game_state.player.units_by_id:
                continue
            if adj_unit.is_worker() and adj_unit.get_cargo_space_used() == 100:
                continue

            # do not transfer to a citytile that can already last for the game
            cityid = game_state.map.get_cityid_of_cell(xx,yy)
            if cityid:
                city: City = game_state.player.cities[cityid]
                if city and city.fuel_needed_for_game < 0:
                    continue

            # if you are on buildable, do not transfer to nonbuildable and noncity
            if tuple(unit.pos) in game_state.buildable_tile_xy_set:
                if (xx,yy) not in game_state.buildable_tile_xy_set and (xx,yy) not in game_state.player_city_tile_xy_set:
                    continue

            print("random transfer", unit.id, unit.pos, "->", adj_unit.id, xx, yy)
            action = unit.transfer(adj_unit.id, unit.cargo.get_most_common_resource(), 2000)
            actions.append(action)
            if annotation:
                actions.append(annotate.text(unit.pos.x, unit.pos.y, annotation))
            actions.append(annotate.line(unit.pos.x, unit.pos.y, adj_unit.pos.x, adj_unit.pos.y))
            unit.cooldown += 2
            break

    # pump and dump
    for unit in player.units:
        unit: Unit = unit
        if not unit.can_act():
            continue
        x,y = tuple(unit.pos)
        if (x,y) in game_state.player_city_tile_xy_set:
            continue
        if (x,y) in game_state.buildable_tile_xy_set:
            continue
        if game_state.convolved_two_opponent_assets_matrix[y,x] < 2:
            continue
        if game_state.convolved_wood_exist_matrix[y,x] <= 1:
            continue
        make_random_transfer(unit, "🟣", True, game_state.player_city_tile_xy_set)

    if initial:
        return actions

    # if moving to a city can let it sustain the night, move into the city
    for unit in player.units:
        unit: Unit = unit
        if not unit.can_act():
            continue
        if game_state.turn%40 < 20:
            continue
        if tuple(unit.pos) not in game_state.buildable_tile_xy_set or not game_state.is_day_time:
            make_random_transfer(unit, "🟢", True, game_state.player_city_tile_xy_set)
        make_random_move_to_city_sustain(unit, "🟢")


    # no cluster rule
    # for unit in player.units:
    #     unit: Unit = unit
    #     if not unit.can_act():
    #         continue
    #     if tuple(unit.pos) not in game_state.player_city_tile_xy_set:
    #         continue
    #     if game_state.player_units_matrix[unit.pos.y,unit.pos.x] > 1:
    #         print("dispersing", unit.id, unit.pos)
    #         make_random_move_to_city(unit, "FY")
    #         make_random_move_to_void(unit, "KD")


    # return to resource to mine
    # for unit in player.units:
    #     unit: Unit = unit
    #     if not unit.can_act():
    #         continue
    #     if tuple(unit.pos) in game_state.convolved_collectable_tiles_xy_set:
    #         continue
    #     if unit.cargo.uranium > 0:
    #         continue
    #     make_random_move_to_collectable(unit, "KC")


    # dump it into a nearby citytile
    for unit in player.units:
        break
        unit: Unit = unit
        if not unit.can_act():
            continue

        # check for full resources
        if unit.get_cargo_space_left() > 4:
            continue
        # if you are in our fortress, dump only if the wood is more than 500
        if game_state.distance_from_floodfill_by_player_city[unit.pos.y, unit.pos.x] >= 2:
            if game_state.wood_amount_matrix[unit.pos.y, unit.pos.x] >= 500:
                print("FA make_random_move_to_city", unit.id)
                make_random_transfer(unit, "FA1", True, game_state.player_city_tile_xy_set)
                make_random_move_to_city(unit, "FA")
        # if you are in a fortress controlled by both players
        elif game_state.distance_from_floodfill_by_either_city[unit.pos.y, unit.pos.x] >= 2:
            print("FB make_random_move_to_city", unit.id)
            make_random_transfer(unit, "FB1", True, game_state.player_city_tile_xy_set)
            make_random_move_to_city(unit, "FB")
        # if you are near opponent assets and you are not on buildable tile
        if game_state.distance_from_opponent_assets[unit.pos.y, unit.pos.x] <= 2:
            if tuple(unit.pos) not in game_state.buildable_tile_xy_set:
                print("FX make_random_move_to_city", unit.id)
                make_random_transfer(unit, "FX1", True, game_state.player_city_tile_xy_set)
                make_random_move_to_city(unit, "FX")


    # make random transfers
    for unit in player.units:
        unit: Unit = unit
        if not unit.can_act():
            continue
        # if unit.get_cargo_space_left() == 0 and unit.is_worker() and game_state.map_resource_count < 500:
        #     actions.append(unit.build_city())
        #     continue
        if unit.get_cargo_space_used() < 10:
            continue
        make_random_transfer(unit, "KT", True, game_state.buildable_tile_xy_set)
        if tuple(unit.pos) in game_state.buildable_tile_xy_set:
            if game_state.distance_from_collectable_resource[unit.pos.y, unit.pos.x] == 1:
                if unit.cargo.get_most_common_resource() == "wood":
                    continue
        make_random_transfer(unit, "KR")


    # no sitting duck not collecting resources
    for unit in player.units:
        unit: Unit = unit
        if not unit.can_act():
            continue
        if tuple(unit.pos) in game_state.convolved_collectable_tiles_xy_set:
            continue
        if unit.fuel_potential == 0:
            if game_state.is_day_time:
                # suicide mission
                make_random_move_to_void(unit, "KS")
        else:
            # move to center so as to consolidate resources
            # make_random_move_to_center(unit, "KP")
            pass


    # make a movement within the city at night, if near the enemy
    for unit in player.units:
        unit: Unit = unit
        if not unit.can_act():
            continue
        if tuple(unit.pos) not in game_state.player_city_tile_xy_set:
            continue
        if game_state.distance_from_opponent_assets[unit.pos.y, unit.pos.x] >= 3:
            continue
        make_random_move_to_city(unit, "MC")


    return actions


def attempt_direction_to(game_state: Game, unit: Unit, target_pos: Position, avoid_opponent_units=False, use_exact=False, DEBUG=False) -> DIRECTIONS:
    if DEBUG: print = __builtin__.print
    else: print = lambda *args: None

    smallest_cost = [2,2,2,2,2]
    closest_dir = DIRECTIONS.CENTER
    closest_pos = unit.pos

    for direction in game_state.dirs:
        newpos = unit.pos.translate(direction, 1)

        cost = [0,0,0,0,0]

        # do not go out of map
        if tuple(newpos) in game_state.xy_out_of_map:
            continue

        # discourage collision among yourself
        # discourage if new position is occupied, not your city tile and not your current position and not your enemy units
        if tuple(newpos) in game_state.occupied_xy_set:
            if tuple(newpos) not in game_state.player_city_tile_xy_set:
                if tuple(newpos) not in game_state.opponent_units_xy_set:
                    if tuple(newpos) != tuple(unit.pos):
                        cost[0] = 3

        if tuple(newpos) in game_state.opponent_units_xy_set:
            if avoid_opponent_units:
                cost[0] = 1
            if tuple(newpos) not in game_state.opponent_units_moveable_xy_set:
                cost[0] = 3

        # discourage going into a city tile if you are carrying substantial wood
        if unit.cargo.wood >= 96:
            if tuple(newpos) in game_state.player_city_tile_xy_set:
                cost[0] = 1

        # discourage going into a city tile if you are carrying substantial wood
        if unit.cargo.wood >= 60:
            if tuple(newpos) in game_state.player_city_tile_xy_set:
                cost[0] = 1

        # no entering opponent citytile
        if tuple(newpos) in game_state.opponent_city_tile_xy_set:
            cost[0] = 4

        # if targeting same cluster, discourage walking on tiles without resources
        targeting_same_cluster = game_state.xy_to_resource_group_id.find(tuple(target_pos)) == game_state.xy_to_resource_group_id.find(tuple(unit.pos))
        if targeting_same_cluster:
            if tuple(newpos) not in game_state.convolved_collectable_tiles_xy_set:
                # unless you have researched uranium or you have some resources
                if not (game_state.player.researched_uranium_projected() or
                        unit.get_cargo_space_used() > 0 or
                        game_state.matrix_player_cities_nights_of_fuel_required_for_night[unit.pos.y, unit.pos.x] < 0):
                    # unless you are very far from opponent
                    if game_state.distance_from_opponent_assets[unit.pos.y,unit.pos.x] < 5:
                        cost[0] = 3

        # discourage going into a fueled city tile if you are carrying substantial coal and uranium
        if unit.cargo.coal + unit.cargo.uranium >= 10:
            if game_state.matrix_player_cities_nights_of_fuel_required_for_game[newpos.y, newpos.x] < 0:
                if tuple(newpos) in game_state.player_city_tile_xy_set:
                    cost[0] = 1

        # path distance as main differentiator
        path_dist = game_state.retrieve_distance(newpos.x, newpos.y, target_pos.x, target_pos.y, use_exact=use_exact)
        cost[1] = path_dist

        # manhattan distance to tie break
        manhattan_dist = (newpos - target_pos)
        cost[2] = manhattan_dist

        # prefer to walk on tiles with resources
        aux_cost = game_state.convolved_collectable_tiles_matrix[newpos.y, newpos.x]
        cost[3] = -min(2,aux_cost)

        # prefer to walk closer to opponent
        aux_cost = game_state.distance_from_opponent_assets[newpos.y, newpos.x]
        cost[4] = aux_cost

        # update decision
        if cost < smallest_cost:
            smallest_cost = cost
            closest_dir = direction
            closest_pos = newpos

        print(newpos, cost)

    if closest_dir != DIRECTIONS.CENTER:
        if tuple(closest_pos) not in game_state.opponent_unit_adjacent_xy_set:
            game_state.occupied_xy_set.discard(tuple(unit.pos))
        if tuple(closest_pos) not in game_state.player_city_tile_xy_set:
            game_state.occupied_xy_set.add(tuple(closest_pos))
        unit.cooldown += 2

    return closest_dir, closest_pos


In [ ]:
%%writefile make_annotations.py
import time
from itertools import chain
from typing import List

import builtins as __builtin__

from lux.game import Game, Mission, Missions, Player, Unit
import lux.annotate as annotate


def annotate_game_state(game_state: Game, DEBUG=False):
    if DEBUG: print = __builtin__.print
    else: print = lambda *args: None

    print("Turn number: ", game_state.turn)
    print("Citytile count: ", game_state.player.city_tile_count)
    print("Unit count: ", len(game_state.player.units))

    if game_state.player_id == 1:
        # reduce clutter for mirror matchup
        return []

    annotations = []

    for city in chain(game_state.player.cities.values(), game_state.opponent.cities.values()):
        for citytile in city.citytiles:
            if city.night_fuel_duration >= game_state.night_turns_left:
                annotation = annotate.circle(citytile.pos.x, citytile.pos.y)
                annotations.append(annotation)
            else:
                annotation = annotate.text(citytile.pos.x, citytile.pos.y, str(city.night_fuel_duration))
                annotations.append(annotation)


    for unit in chain(game_state.player.units, game_state.opponent.units):
        if unit.cargo.get_shorthand():
            annotation = annotate.text(unit.pos.x, unit.pos.y, unit.cargo.get_shorthand())
            annotations.append(annotation)

    annotation = annotate.text(int(game_state.resource_median.x), int(game_state.resource_median.y), "MD")
    annotations.append(annotation)
    annotation = annotate.text(int(game_state.resource_mean.x), int(game_state.resource_mean.y), "ME")
    annotations.append(annotation)
    annotation = annotate.text(int(game_state.player_unit_median.x), int(game_state.player_unit_median.y), "PD")
    annotations.append(annotation)
    annotation = annotate.text(int(game_state.player_city_median.x), int(game_state.player_city_median.y), "PE")
    annotations.append(annotation)

    # you can also read the pickled game_state and print its attributes
    return annotations


def annotate_missions(game_state: Game, missions: Missions, DEBUG=False):
    if DEBUG: print = __builtin__.print
    else: print = lambda *args: None

    print("Missions")
    print(missions)
    # you can also read the pickled missions and print its attributes

    annotations: List[str] = []
    player: Player = game_state.player

    for unit_id, mission in missions.items():
        mission: Mission = mission
        unit: Unit = player.units_by_id[unit_id]

        annotation = annotate.line(unit.pos.x, unit.pos.y, mission.target_position.x, mission.target_position.y)
        annotations.append(annotation)

        # if mission.target_action and mission.target_action.split(" ")[0] == "bcity":
        #     annotation = annotate.x(mission.target_position.x, mission.target_position.y)
        #     annotations.append(annotation)
        # else:
        #     annotation = annotate.circle(mission.target_position.x, mission.target_position.y)
        #     annotations.append(annotation)

    annotation = annotate.sidetext("Unit Count: {}-{} Citytiles: {}-{} Groups: {}/{} Runtime: {:.3f}".format(
        len(game_state.player.units), len(game_state.opponent.units),
        len(game_state.player_city_tile_xy_set), len(game_state.opponent_city_tile_xy_set),
        game_state.targeted_cluster_count, game_state.xy_to_resource_group_id.get_group_count(),
        time.time() - game_state.compute_start_time))
    annotations.append(annotation)

    return annotations


def annotate_movements(game_state: Game, actions_by_units: List[str]):
    annotations = []
    dirs = game_state.dirs
    d5 = game_state.dirs_dxdy

    for action_by_units in actions_by_units:
        if action_by_units[:2] != "m ":
            continue
        unit_id, dir = action_by_units.split(" ")[1:]
        unit = game_state.player.units_by_id[unit_id]
        x, y = unit.pos.x, unit.pos.y
        dx, dy = d5[dirs.index(dir)]
        annotation = annotate.line(x, y, x+dx, y+dy)
        annotations.append(annotation)

    return annotations


def filter_cell_annotations(actions: List[str], game_state: Game):
    annotated_cell_xy_set = set()
    filtered_actions: List[str] = []
    for action in actions:
        if action[:2] == "m " and action[-2:] == ' c':
            continue
        instruction, *info = action.split()
        if instruction == "dt":
            if (info[0],info[1]) in annotated_cell_xy_set:
                continue
            annotated_cell_xy_set.add((info[0],info[1]))
        filtered_actions.append(action)
    for unit in game_state.player.units:
        if unit.cooldown < 1:
            no_action = unit.move("c")
            filtered_actions.append(no_action)
            filtered_actions.append(annotate.text(unit.pos.x, unit.pos.y, "🟤"))
    return filtered_actions


In [ ]:
%%writefile heuristics.py
# contains designed heuristics
# which could be fine tuned
import math
import time

import numpy as np
import builtins as __builtin__

from typing import Dict
from lux import annotate
from lux import game

from lux.game import Game, Unit
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_position import Position
from lux.game_constants import GAME_CONSTANTS


def find_best_cluster(game_state: Game, unit: Unit, DEBUG=False, explore=False, require_empty_target=False, ref_pos:Position=None):

    if DEBUG: print = __builtin__.print
    else: print = lambda *args: None

    # for debugging
    score_matrix_wrt_pos = game_state.init_matrix()

    # default response is not to move
    best_position = unit.pos
    best_cell_value = [0,0,0,0]
    cluster_annotation = []

    if time.time() - game_state.compute_start_time > 3:
        # running out of time
        return best_position, best_cell_value, cluster_annotation

    # if at night, if near enemy or almost dawn, if city is going to die, if staying can keep the city alive
    if not game_state.is_day_time:
        cityid = game_state.map.get_cityid_of_cell(unit.pos.x, unit.pos.y)
        if cityid:
            city = game_state.player.cities[cityid]
            if game_state.distance_from_opponent_assets[unit.pos.y,unit.pos.x] <= 2 or city.fuel_needed_for_night <= len(city.citytiles) * 120:
                if city.fuel_needed_for_night > 0:
                    if city.fuel_needed_for_night - game_state.fuel_collection_rate[unit.pos.y, unit.pos.x] * game_state.turns_to_dawn <= 0:
                        unit.cooldown += 1
                        best_cell_value = [10**9,0,0,0]
                        print("staying SU", unit.id, unit.pos)
                        annotation = annotate.text(unit.pos.x, unit.pos.y, "SU")
                        cluster_annotation.append(annotation)

    # anticipate pump and dump
    if tuple(unit.pos) in game_state.player_city_tile_xy_set:
        for dy,dx in game_state.dirs_dxdy[:-1]:
            xx,yy = unit.pos.x+dx, unit.pos.y+dy
            if (xx,yy) not in game_state.player.units_by_xy:
                continue
            adj_unit: Unit = game_state.player.units_by_xy[xx,yy]
            if game_state.convolved_opponent_assets_matrix[yy,xx] < 2:
                if game_state.convolved_opponent_assets_matrix[unit.pos.y,unit.pos.x] < 2:
                    continue
            if game_state.convolved_wood_exist_matrix[yy,xx] <= 1:
                continue
            if (xx,yy) in game_state.buildable_tile_xy_set:
                continue
            unit.cooldown += 1
            print("staying SP", unit.id, unit.pos)
            best_cell_value = [10**9,0,0,0]
            annotation = annotate.text(unit.pos.x, unit.pos.y, "SP")
            cluster_annotation.append(annotation)

    # anticipate ejection
    if tuple(unit.pos) in game_state.player_city_tile_xy_set and False:
      if game_state.xy_to_resource_group_id.get_point(tuple(unit.pos)) <= 3:
        for dy,dx in game_state.dirs_dxdy[:-1]:
            xx,yy = unit.pos.x+dx, unit.pos.y+dy
            if (xx,yy) not in game_state.player.units_by_xy:
                continue
            adj_unit: Unit = game_state.player.units_by_xy[xx,yy]
            if int(adj_unit.cooldown) != 1:
                continue
            if game_state.convolved_wood_exist_matrix[yy,xx] < 1:
                continue
            unit.cooldown += 1
            print("staying SX", unit.id, unit.pos)
            best_cell_value = [10**9,0,0,0]
            annotation = annotate.text(unit.pos.x, unit.pos.y, "SX")
            cluster_annotation.append(annotation)

    # only consider other cluster if the current cluster has more than one agent mining
    consider_different_cluster = False
    # must consider other cluster if the current cluster has more agent than tiles
    consider_different_cluster_must = explore

    # calculate how many resource tiles and how many units on the current cluster
    current_leader = game_state.xy_to_resource_group_id.find(tuple(unit.pos))
    units_mining_on_current_cluster = game_state.resource_leader_to_locating_units[current_leader] & game_state.resource_leader_to_targeting_units[current_leader]
    resource_size_of_current_cluster = game_state.xy_to_resource_group_id.get_point(current_leader)
    if game_state.distance_from_opponent_assets[unit.pos.y, unit.pos.x] > 10:
        if resource_size_of_current_cluster > 1:
            resource_size_of_current_cluster = resource_size_of_current_cluster//2

    # only consider other cluster if another unit is targeting and mining in the current cluster
    if len(units_mining_on_current_cluster - set([unit.id])) >= 1:
        consider_different_cluster = True

    # if you are in a barren field you must consider a different cluster
    if tuple(unit.pos) not in game_state.convolved_collectable_tiles_xy_set:
        consider_different_cluster_must = True

    if len(units_mining_on_current_cluster) >= resource_size_of_current_cluster:
        # must consider if you have more than enough workers in the current cluster
        consider_different_cluster_must = True

    print("finding best cluster for", unit.id, unit.pos, consider_different_cluster, consider_different_cluster_must)

    best_citytile_of_cluster: Dict = dict()
    target_bonus_for_current_cluster_logging = -999

    for y in game_state.y_iteration_order:
        for x in game_state.x_iteration_order:

            # what not to target
            if (x,y) in game_state.targeted_for_building_xy_set:
                continue
            if (x,y) in game_state.opponent_city_tile_xy_set:
                continue
            if (x,y) in game_state.player_city_tile_xy_set:
                continue

            if ref_pos:
                if abs(ref_pos.x - x) + abs(ref_pos.y - y) < abs(unit.pos.x - x) + abs(unit.pos.y - y):
                    continue

            # allow multi targeting of uranium mines
            if game_state.convolved_uranium_exist_matrix[y,x] == 0 or \
                not game_state.player.researched_uranium_projected() or \
                    game_state.matrix_player_cities_nights_of_fuel_required_for_night[y,x] <= 0:
                if (x,y) in game_state.targeted_xy_set:
                    continue

            if require_empty_target and len(units_mining_on_current_cluster) <= 2:
                continue

            distance = game_state.retrieve_distance(unit.pos.x, unit.pos.y, x, y)

            # cluster targeting logic

            # target bonus should have the same value for the entire cluster
            target_bonus = 1
            target_leader = game_state.xy_to_resource_group_id.find((x,y))
            if consider_different_cluster or consider_different_cluster_must:
                # if the target is a cluster and not the current cluster
                if target_leader:

                    units_targeting_or_mining_on_target_cluster = \
                        game_state.resource_leader_to_locating_units[target_leader] | \
                        game_state.resource_leader_to_targeting_units[target_leader]

                    if require_empty_target and units_targeting_or_mining_on_target_cluster:
                        continue
                    resource_size_of_target_cluster = game_state.xy_to_resource_group_id.get_point(target_leader)

                    # target bonus depends on how many resource tiles and how many units that are mining or targeting
                    target_bonus = resource_size_of_target_cluster/\
                                   (1 + len(units_targeting_or_mining_on_target_cluster))

                    # avoid targeting overpopulated clusters
                    if len(units_targeting_or_mining_on_target_cluster) > resource_size_of_target_cluster:
                        target_bonus = target_bonus * 0.1

                    # if none of your units is targeting the cluster and definitely reachable
                    if len(units_targeting_or_mining_on_target_cluster) == 0:
                        if distance <= game_state.distance_from_opponent_assets[y,x]:
                            target_bonus = target_bonus * 10

                    # discourage targeting depending are you the closest unit to the resource
                    distance_bonus = max(1,game_state.distance_from_player_assets[y,x])/max(1,distance)

                    if require_empty_target and distance_bonus < 1:
                        continue

                    if consider_different_cluster_must:
                        distance_bonus = max(1/2, distance_bonus)

                    target_bonus = target_bonus * distance_bonus**2

                    if distance_bonus == 1:
                        # extra bonus if you are closest to the target
                        target_bonus = target_bonus * 10

                    # travel penalty
                    target_bonus = target_bonus / math.log(4 + game_state.xy_to_resource_group_id.get_dist_from_player((x,y),), 2)

                    # if targeted cluster is much closer to enemy, do not target if cannot survive the night
                    # resources is required for invasion
                    if game_state.distance_from_opponent_assets[y,x] + 5 < \
                       game_state.xy_to_resource_group_id.get_dist_from_player((x,y),):
                        if unit.night_turn_survivable < 10:
                            target_bonus = target_bonus * 0.01

                    # slightly discourage targeting clusters closer to enemy
                    if game_state.xy_to_resource_group_id.get_dist_from_opponent((x,y),) < \
                       game_state.xy_to_resource_group_id.get_dist_from_player((x,y),):
                        target_bonus = target_bonus * 0.9

            if target_leader and target_leader == current_leader:
                # if targeting same cluster do not move more than five
                if distance > 5:
                    continue

            if consider_different_cluster_must and target_leader != current_leader:
                # enforce targeting of other clusters
                target_bonus = target_bonus * 10

            if not consider_different_cluster_must and target_leader == current_leader:
                target_bonus = target_bonus * 2

            # only target cells where you can collect resources
            if game_state.convolved_collectable_tiles_matrix_projected[y,x] == 0:
                continue

            if unit.night_turn_survivable < 10:
                if game_state.convolved_collectable_tiles_matrix[y,x] == 0:
                    continue

            # identation to retain commit history
            if True:
                # do not plan overnight missions if you are the only unit mining
                if tuple(unit.pos) in game_state.convolved_collectable_tiles_xy_set:
                    if len(units_mining_on_current_cluster) <= 1 and distance > 15:
                        continue

                # estimate target score
                if distance <= unit.travel_range:
                    cell_value = [target_bonus,
                                  - game_state.distance_from_floodfill_by_empty_tile[y,x],
                                  - game_state.distance_from_resource_median[y,x]
                                  - distance - game_state.distance_from_opponent_assets[y,x]
                                  - distance + game_state.distance_from_player_unit_median[y,x],
                                  - distance - game_state.opponent_units_matrix[y,x] * 2]

                    # penalty on parameter preference
                    # if not collectable and not buildable, penalise
                    if (x,y) not in game_state.collectable_tiles_xy_set and (x,y) not in game_state.buildable_tile_xy_set:
                        cell_value[1] -= 1

                    # prefer to mine advanced resources faster
                    if unit.get_cargo_space_left() > 8:
                        if game_state.player.researched_coal_projected():
                            cell_value[1] += 2*game_state.convolved_coal_exist_matrix[y,x]
                        if game_state.player.researched_uranium_projected():
                            cell_value[1] += 2*game_state.convolved_uranium_exist_matrix[y,x]

                    # if mining advanced resource, stand your ground unless there is a direct path
                    if game_state.convolved_coal_exist_matrix[unit.pos.y,unit.pos.x] or game_state.convolved_uranium_exist_matrix[unit.pos.y,unit.pos.x]:
                        if distance > abs(unit.pos.x - x) + abs(unit.pos.y - y):
                            continue

                    # discourage if the target is one unit closer to the enemy, in the early game
                    # specific case to avoid this sort of targeting (A -> X)
                    #    X
                    # WABW
                    # WWWW
                    if game_state.distance_from_opponent_assets[y,x] + 1 == game_state.distance_from_player_units[y,x]:
                        if game_state.turn < 80:
                            cell_value[2] -= 2

                    # for first target prefer B over A
                    #   X
                    # BWWW
                    #  WWW
                    #  AX
                    if game_state.distance_from_opponent_assets[y,x] == 1 and game_state.distance_from_player_assets[y,x] > 2:
                        if game_state.turn < 1:
                            cell_value[2] -= 2

                    # discourage if you are in the citytile, and you are targeting the location beside you with one wood side
                    # specific case to avoid this sort of targeting (A -> X), probably encourage (A -> Z) or (A -> Y)
                    #
                    #   AX
                    #  ZWWY
                    if tuple(unit.pos) in game_state.player_city_tile_xy_set:
                        if Position(x,y) - unit.pos == 1:
                            if game_state.convolved_wood_exist_matrix[y,x] == 1 and game_state.resource_collection_rate[y,x] == 20:
                                if game_state.distance_from_opponent_units[y,x] > 2:
                                    cell_value[2] -= 5


                    # if more than 20 uranium do not target a wood cluster so that it can home
                    if unit.cargo.uranium > 20:
                        if game_state.convolved_wood_exist_matrix[y,x]*20 == game_state.resource_collection_rate[y,x]:
                            cell_value[0] = -1

                    # for debugging
                    score_matrix_wrt_pos[y,x] = cell_value[2]

                    # update best target
                    if cell_value > best_cell_value:
                        best_cell_value = cell_value
                        best_position = Position(x,y)

                    if target_leader not in best_citytile_of_cluster:
                        best_citytile_of_cluster[target_leader] = (cell_value,x,y)
                    if (cell_value,x,y) > best_citytile_of_cluster[target_leader]:
                        best_citytile_of_cluster[target_leader] = (cell_value,x,y)

                    if target_leader == current_leader:
                        target_bonus_for_current_cluster_logging = max(target_bonus_for_current_cluster_logging, target_bonus)

    # annotate if target bonus is more than one
    if best_cell_value[0] > target_bonus_for_current_cluster_logging > -999:
        for cell_value,x,y in sorted(best_citytile_of_cluster.values())[:10]:
            annotation = annotate.text(x,y,f"{int(cell_value[0])}")
            cluster_annotation.append(annotation)
            annotation = annotate.line(unit.pos.x,unit.pos.y,x,y)
            cluster_annotation.append(annotation)

    # for debugging
    game_state.heuristics_from_positions[tuple(unit.pos)] = score_matrix_wrt_pos

    return best_position, best_cell_value, cluster_annotation


In [ ]:
%%writefile main.py
from typing import Dict
import sys
from agent import agent
if __name__ == "__main__":

    def read_input():
        """
        Reads input from stdin
        """
        try:
            return input()
        except EOFError as eof:
            raise SystemExit(eof)
    step = 0
    class Observation(Dict[str, any]):
        def __init__(self, player=0) -> None:
            self.player = player
            # self.updates = []
            # self.step = 0
    observation = Observation()
    observation["updates"] = []
    observation["step"] = 0
    player_id = 0
    while True:
        inputs = read_input()
        observation["updates"].append(inputs)

        if inputs == "D_DONE":
            if step == 0:  # the codefix
                player_id = int(observation["updates"][0])
                observation.player = player_id
                observation["player"] = player_id
                observation["width"], observation["height"] = map(int, observation["updates"][1].split())
            actions = agent(observation, None)
            observation["updates"] = []
            step += 1
            observation["step"] = step
            print(",".join(actions))
            print("D_FINISH")


# Upgraded Game Kit
The game kit has been edited to include more features for the agent to make decisions on.


In [ ]:
%%writefile lux/game.py
import heapq, time
from collections import defaultdict, deque
from typing import DefaultDict, Dict, List, Tuple, Set
from datetime import datetime
import builtins as __builtin__

import numpy as np

from .constants import Constants
from .game_map import GameMap, RESOURCE_TYPES
from .game_objects import Player, Unit, City
from .game_position import Position
from .game_constants import GAME_CONSTANTS

INPUT_CONSTANTS = Constants.INPUT_CONSTANTS


class Observation(Dict[str, any]):
    def __init__(self, player=0) -> None:
        self.player = player
        # self.updates = []
        # self.step = 0


class Mission:
    def __init__(self, unit_id: str, target_position: Position, target_action: str = "", details: str = "", delays=99):
        self.target_position: Position = target_position
        self.target_action: str = target_action
        self.unit_id: str = unit_id
        self.delays: int = 2*delays
        self.details: str = details  # block deletion of mission if no resource
        # [TODO] some expiry date for each mission

    def __str__(self):
        return " ".join([str(self.target_position), self.target_action, str(self.delays)])


class Missions(defaultdict):
    def __init__(self):
        self: DefaultDict[str, Mission] = defaultdict(Mission)

    def add(self, mission: Mission):
        self[mission.unit_id] = mission

    def __str__(self):
        return " | ".join([unit_id + " " + str(mission) for unit_id,mission in self.items()])

    def get_targets(self):
        return [mission.target_position for unit_id, mission in self.items()]

    def get_target_of_unit(self, unit_id):
        return {unit_id: mission.target_position for unit_id, mission in self.items()}[unit_id]

    def get_targets_and_actions(self):
        return [(mission.target_position, mission.target_action) for unit_id, mission in self.items()]

    def reset_missions(self, research_points, convolved_coal_exist_matrix, convolved_uranium_exist_matrix):
        for unit_id in list(self.keys()):
            x,y = tuple(self[unit_id].target_position)
            if research_points >= 200:
                if convolved_uranium_exist_matrix[y,x] == 0:
                    del self[unit_id]
                    continue
            elif research_points >= 50:
                if convolved_coal_exist_matrix[y,x] == 0:
                    del self[unit_id]
                    continue

class DisjointSet:
    def __init__(self):
        self.parent = {}
        self.sizes = defaultdict(int)
        self.points = defaultdict(int)  # 1 point for wood, 3 point for coal, 5 point for uranium
        self.tiles = defaultdict(int)  # 1 point for all resource
        self.citytiles = defaultdict(int)  # 1 point for citytile next to cluster
        self.dist_from_player = defaultdict(int)  # closest distance from player
        self.dist_from_opponent = defaultdict(int)  # closest distance from player
        self.num_sets = 0

    def find(self, a, point=0, tile=0, citytile=0):
        assert type(a) == tuple
        if a not in self.parent:
            self.parent[a] = a
            self.sizes[a] += 1
            self.points[a] += point
            self.tiles[a] += tile
            self.citytiles[a] += citytile
            self.num_sets += 1
        acopy = a
        while a != self.parent[a]:
            a = self.parent[a]
        while acopy != a:
            self.parent[acopy], acopy = a, self.parent[acopy]
        return a

    def union(self, a, b):
        assert type(a) == tuple
        assert type(b) == tuple
        a, b = self.find(a), self.find(b)
        if a != b:
            # if self.sizes[a] < self.sizes[b]:
            #     a, b = b, a

            self.num_sets -= 1
            self.parent[b] = a
            self.sizes[a] += self.sizes[b]
            self.points[a] += self.points[b]
            self.tiles[a] += self.tiles[b]
            self.citytiles[a] += self.citytiles[b]

    def get_size(self, a):
        assert type(a) == tuple
        return self.sizes[self.find(a)]

    def get_point(self, a):
        assert type(a) == tuple
        return self.points[self.find(a)]

    def get_tiles(self, a):
        assert type(a) == tuple
        return self.tiles[self.find(a)]

    def get_citytiles(self, a):
        assert type(a) == tuple
        return self.citytiles[self.find(a)]

    def get_dist_from_player(self, a):
        assert type(a) == tuple
        if self.find(a) not in self.dist_from_player:
            return 100
        return self.dist_from_player[self.find(a)]

    def get_dist_from_opponent(self, a):
        assert type(a) == tuple
        if self.find(a) not in self.dist_from_opponent:
            return 100
        return self.dist_from_opponent[self.find(a)]

    def get_groups(self):
        groups = defaultdict(list)
        for element in self.parent:
            leader = self.find(element)
            if leader:
                groups[leader].append(element)
        return groups

    def get_groups_sorted_by_citytile_size(self):
        groups = defaultdict(list)
        for element in self.parent:
            leader = self.find(element)
            if leader:
                groups[leader].append(element)
        leaders = list(groups.keys())
        leaders.sort(key=lambda leader: (self.get_citytiles(leader), self.get_tiles(leader)), reverse=True)
        return [groups[leader] for leader in leaders if self.get_point(leader) > 0]

    def get_group_count(self):
        return sum(self.points[leader] > 1 for leader in self.get_groups().keys())


class Game:

    # counted from the time after the objects are saved to disk
    compute_start_time = -1

    def _initialize(self, messages):
        """
        initialize state
        """
        self.player_id: int = int(messages[0])
        self.turn: int = -1
        # get some other necessary initial input
        mapInfo = messages[1].split(" ")
        self.map_width: int = int(mapInfo[0])
        self.map_height: int = int(mapInfo[1])
        self.map: GameMap = GameMap(self.map_width, self.map_height)
        self.players: List[Player] = [Player(0), Player(1)]

        self.y_order_coefficient = 1
        self.x_order_coefficient = 1
        self.x_iteration_order = list(range(self.map_width))
        self.y_iteration_order = list(range(self.map_height))
        self.dirs: List = [
            Constants.DIRECTIONS.NORTH,
            Constants.DIRECTIONS.EAST,
            Constants.DIRECTIONS.SOUTH,
            Constants.DIRECTIONS.WEST,
            Constants.DIRECTIONS.CENTER
        ]
        self.dirs_dxdy: List = [(0,-1), (1,0), (0,1), (-1,0), (0,0)]
        self.units_expected_to_act: Set[Tuple] = set()


    def fix_iteration_order(self):
        '''
        Fix iteration order at initisation to allow moves to be symmetric
        '''
        assert len(self.player.cities) == 1
        assert len(self.opponent.cities) == 1
        px,py = tuple(list(self.player.cities.values())[0].citytiles[0].pos)
        ox,oy = tuple(list(self.opponent.cities.values())[0].citytiles[0].pos)

        flipping = False
        self.y_order_coefficient = 1
        self.x_order_coefficient = 1

        if px == ox:
            if py < oy:
                flipping = True
                self.y_iteration_order = self.y_iteration_order[::-1]
                self.y_order_coefficient = -1
                idx1, idx2 = 0,2
        elif py == oy:
            if px < ox:
                flipping = True
                self.x_iteration_order = self.x_iteration_order[::-1]
                self.x_order_coefficient = -1
                idx1, idx2 = 1,3
        else:
            assert False

        if flipping:
            self.dirs[idx1], self.dirs[idx2] = self.dirs[idx2], self.dirs[idx1]
            self.dirs_dxdy[idx1], self.dirs_dxdy[idx2] = self.dirs_dxdy[idx2], self.dirs_dxdy[idx1]


    def _end_turn(self):
        print("D_FINISH")


    def _reset_player_states(self):
        self.players[0].units = []
        self.players[0].cities = {}
        self.players[0].city_tile_count = 0
        self.players[1].units = []
        self.players[1].cities = {}
        self.players[1].city_tile_count = 0

        self.player: Player = self.players[self.player_id]
        self.opponent: Player = self.players[1 - self.player_id]


    def _update(self, messages):
        """
        update state
        """
        self.map = GameMap(self.map_width, self.map_height)
        self.turn += 1
        self._reset_player_states()

        # [TODO] Use constants here
        self.night_turns_left = (360 - self.turn)//40 * 10 + min(10, (360 - self.turn)%40)

        self.turns_to_night = (30 - self.turn)%40
        self.turns_to_night = 0 if self.turns_to_night > 30 else self.turns_to_night

        self.turns_to_dawn = (40 - self.turn%40)
        self.turns_to_dawn = 0 if self.turns_to_dawn > 10 else self.turns_to_dawn

        self.is_day_time = self.turns_to_dawn == 0

        for update in messages:
            if update == "D_DONE":
                break
            strs = update.split(" ")
            input_identifier = strs[0]

            if input_identifier == INPUT_CONSTANTS.RESEARCH_POINTS:
                team = int(strs[1])   # probably player_id
                self.players[team].research_points = int(strs[2])

            elif input_identifier == INPUT_CONSTANTS.RESOURCES:
                r_type = strs[1]
                x = int(strs[2])
                y = int(strs[3])
                amt = int(float(strs[4]))
                self.map._setResource(r_type, x, y, amt)

            elif input_identifier == INPUT_CONSTANTS.UNITS:
                unittype = int(strs[1])
                team = int(strs[2])
                unitid = strs[3]
                x = int(strs[4])
                y = int(strs[5])
                cooldown = float(strs[6])
                wood = int(strs[7])
                coal = int(strs[8])
                uranium = int(strs[9])
                unit = Unit(team, unittype, unitid, x, y, cooldown, wood, coal, uranium)
                self.players[team].units.append(unit)
                self.map.get_cell(x, y).unit = unit

            elif input_identifier == INPUT_CONSTANTS.CITY:
                team = int(strs[1])
                cityid = strs[2]
                fuel = float(strs[3])
                lightupkeep = float(strs[4])
                self.players[team].cities[cityid] = City(team, cityid, fuel, lightupkeep, self.night_turns_left)

            elif input_identifier == INPUT_CONSTANTS.CITY_TILES:
                team = int(strs[1])
                cityid = strs[2]
                x = int(strs[3])
                y = int(strs[4])
                cooldown = float(strs[5])
                city = self.players[team].cities[cityid]
                citytile = city._add_city_tile(x, y, cooldown)
                self.map.get_cell(x, y).citytile = citytile
                self.players[team].city_tile_count += 1

            elif input_identifier == INPUT_CONSTANTS.ROADS:
                x = int(strs[1])
                y = int(strs[2])
                road = float(strs[3])
                self.map.get_cell(x, y).road = road

        # create indexes to refer to unit by id
        self.player.make_index_units_by_id()
        self.opponent.make_index_units_by_id()

        if self.turn > 0:
            for city in self.player.cities.values():
                city.citytiles.sort(key=lambda city_tile:(
                    city_tile.pos.x * self.x_order_coefficient,
                    city_tile.pos.y * self.y_order_coefficient))

            for city in self.opponent.cities.values():
                city.citytiles.sort(key=lambda city_tile:(
                    city_tile.pos.x * self.x_order_coefficient,
                    city_tile.pos.y * self.y_order_coefficient))

        # rotate iteration order
        if self.turn%4 != 0:
            self.dirs[3], self.dirs[0:3] = self.dirs[0], self.dirs[1:4]
            self.dirs_dxdy[3], self.dirs_dxdy[0:3] = self.dirs_dxdy[0], self.dirs_dxdy[1:4]

        self.unit_ids_with_missions_assigned_this_turn: Set = set()

        for unit in self.player.units:
            unit.compute_travel_range((self.turns_to_night, self.turns_to_dawn, self.is_day_time),)
        for unit in self.opponent.units:
            unit.compute_travel_range((self.turns_to_night, self.turns_to_dawn, self.is_day_time),)


    def calculate_features(self, missions: Missions):

        # load constants into object
        self.wood_fuel_rate = GAME_CONSTANTS["PARAMETERS"]["RESOURCE_TO_FUEL_RATE"][RESOURCE_TYPES.WOOD.upper()]
        self.wood_collection_rate = GAME_CONSTANTS["PARAMETERS"]["WORKER_COLLECTION_RATE"][RESOURCE_TYPES.WOOD.upper()]
        self.coal_fuel_rate = GAME_CONSTANTS["PARAMETERS"]["RESOURCE_TO_FUEL_RATE"][RESOURCE_TYPES.COAL.upper()]
        self.coal_collection_rate = GAME_CONSTANTS["PARAMETERS"]["WORKER_COLLECTION_RATE"][RESOURCE_TYPES.COAL.upper()]
        self.uranium_fuel_rate = GAME_CONSTANTS["PARAMETERS"]["RESOURCE_TO_FUEL_RATE"][RESOURCE_TYPES.URANIUM.upper()]
        self.uranium_collection_rate = GAME_CONSTANTS["PARAMETERS"]["WORKER_COLLECTION_RATE"][RESOURCE_TYPES.URANIUM.upper()]

        # update matrices
        self.calculate_matrix()
        self.calculate_resource_matrix()
        self.calculate_resource_groups()
        self.calculate_distance_matrix()

        # when to use rules
        for unit in self.player.units:
            x,y = tuple(unit.pos)
            if self.player.researched_uranium() and unit.cargo.wood == 100:
                unit.use_rule_base = False
                continue
            if self.turn >= 348:
               unit.use_rule_base = True
               continue
            if unit.cargo.uranium > 20 and (x,y) not in self.convolved_collectable_tiles_xy_set:
                unit.use_rule_base = True
                continue
            # if unit.cargo.coal >= 50:
            #     unit.use_rule_base = True
            #     continue
            if self.distance_from_collectable_resource_projected[y,x] > 6:
                unit.use_rule_base = True
                continue
            if self.distance_from_wood_tile[y,x] < 4:
                unit.use_rule_base = False
                continue

        self.player_unit_can_act_count = 1
        for unit in self.player.units:
            if unit.can_act() and unit.use_rule_base:
                self.player_unit_can_act_count += 1

        self.sinking_cities_xy_set = set()
        for city in self.player.cities.values():
            if self.is_day_time:
                continue

            collection_rates = []
            adjacent_and_residing_units = set()
            for citytile in city.citytiles:
                collection_rates.append(self.fuel_collection_rate[citytile.pos.y, citytile.pos.x])
                for dx,dy in self.dirs_dxdy:
                    xx, yy = citytile.pos.x+dx, citytile.pos.y+dy
                    if (xx, yy) in self.player.units_by_xy:
                        adj_unit = self.player.units_by_xy[xx,yy]
                        if adj_unit.can_act():
                            adjacent_and_residing_units.add(adj_unit.id)
            collection_rates.sort(reverse=True)

            maximum_collection = sum(collection_rates[:len(adjacent_and_residing_units)])
            maximum_injection = 0
            for adj_unit_id in adjacent_and_residing_units:
                adj_unit = self.player.units_by_id[adj_unit_id]
                maximum_injection += adj_unit.fuel_potential

            if city.fuel + maximum_collection + maximum_injection < city.get_light_upkeep():
                for citytile in city.citytiles:
                    self.sinking_cities_xy_set.add((citytile.pos.x, citytile.pos.y))
                    self.occupied_xy_set.add((citytile.pos.x, citytile.pos.y))

        # if there are cities beside coal or uranium, stop producing units and do research, and encourage building cities
        self.worker_production_ban = False
        for city in self.player.cities.values():
            city_beside_coal = False
            city_beside_uranium = False

        # 1 transform 10 ms
        # 4 transform 25 ms
        # 8 transform 45 ms

        allowed_transforms = 1500//5
        self.number_of_transforms = max(1, min(8, allowed_transforms//self.player_unit_can_act_count))

        # # gating
        # for unit in self.player.units:
        #     unit.use_rule_base = False


        # places to avoid building workers
        self.avoid_building_workers_xy_set: Set = set()
        for x in self.x_iteration_order:
            for y in self.y_iteration_order:
                if self.turn > 350:
                    continue
                if (x,y) not in self.player_city_tile_xy_set:
                    continue
                if not self.player.researched_uranium():
                    continue
                if self.distance_from_opponent_assets[y,x] < 3:
                    continue
                if self.distance_from_player_units[y,x] >= 3:
                    continue
                if self.distance_from_player_units[y,x] <= 1 and self.distance_from_wood_tile[y,x] < 3:
                    self.avoid_building_workers_xy_set.add((x,y),)
                    continue
                for dx,dy in self.dirs_dxdy:
                    xx, yy = x+dx, y+dy
                    if not (0 <= xx < self.map_width and 0 <= yy < self.map_height):
                        continue
                    if 0 < self.wood_amount_matrix[yy,xx] < 500:
                        self.avoid_building_workers_xy_set.add((x,y),)
                        break

        # place and time to avoid building citytiles
        self.avoid_building_citytiles_xy_set: Set = set()
        for x in self.x_iteration_order:
            for y in self.y_iteration_order:
                if self.turn%40 != 30:
                    continue
                if self.distance_from_player_citytiles[y,x] <= 1:
                    continue
                if self.fuel_collection_rate[y,x] >= 23:
                    continue
                self.avoid_building_citytiles_xy_set.add((x,y), )


        self.repopulate_targets(missions)

        self.player.units.sort(key=lambda unit: (
            unit.get_cargo_space_left() > 0,
            tuple(unit.pos) not in self.player_city_tile_xy_set,
            self.distance_from_opponent_assets[unit.pos.y,unit.pos.x],
            self.distance_from_resource_median[unit.pos.y,unit.pos.x],
            unit.pos.x*self.x_order_coefficient,
            unit.pos.y*self.y_order_coefficient,
            unit.encode_tuple_for_cmp()))


        self.citytiles_with_new_units_xy_set: Set = set()
        self.heuristics_from_positions: Dict = dict()

        self.units_did_not_act: Set = set(unit_id for unit_id in self.units_expected_to_act)
        for unit in self.player.units:
            if unit.can_act():
                self.units_expected_to_act.add(unit.id)
        self.units_did_not_act = self.units_did_not_act & self.units_expected_to_act

        update_mission_delay(self, missions)


    def init_matrix(self, default_value=0):
        # [TODO] check if order of map_height and map_width is correct
        return np.full((self.map_height,self.map_width), default_value)


    def calculate_matrix(self):

        # amount of resources left on the tile
        self.wood_amount_matrix = self.init_matrix()
        self.coal_amount_matrix = self.init_matrix()
        self.uranium_amount_matrix = self.init_matrix()
        self.all_resource_amount_matrix = self.init_matrix()

        self.player_city_tile_matrix = self.init_matrix()
        self.opponent_city_tile_matrix = self.init_matrix()

        self.player_units_matrix = self.init_matrix()
        self.opponent_units_matrix = self.init_matrix()

        # if there is nothing on tile
        self.empty_tile_matrix = self.init_matrix()

        # if you can build on tile (a unit may be on the tile)
        self.buildable_tile_matrix = self.init_matrix()
        self.probably_buildable_tile_matrix = self.init_matrix()
        self.preferred_buildable_tile_matrix = self.init_matrix()

        self.road_level_matrix = self.init_matrix()

        for y in self.y_iteration_order:
            for x in self.x_iteration_order:
                cell = self.map.get_cell(x, y)

                self.road_level_matrix[y,x] = cell.road

                is_empty = True
                is_buildable = True

                if cell.unit:
                    is_empty = False
                    # unit counting method implemented later
                    # cell.unit only contain one unit even though multiple units can stay in citytile

                if cell.has_resource():
                    is_empty = False
                    is_buildable = False
                    if cell.resource.type == RESOURCE_TYPES.WOOD:
                        self.wood_amount_matrix[y,x] += cell.resource.amount
                    if cell.resource.type == RESOURCE_TYPES.COAL:
                        self.coal_amount_matrix[y,x] += cell.resource.amount
                    if cell.resource.type == RESOURCE_TYPES.URANIUM:
                        self.uranium_amount_matrix[y,x] += cell.resource.amount
                    self.all_resource_amount_matrix[y,x] += cell.resource.amount

                elif cell.citytile:
                    is_empty = False
                    is_buildable = False
                    if cell.citytile.team == self.player_id:
                        self.player_city_tile_matrix[y,x] += 1
                    else:   # city tile belongs to opponent
                        self.opponent_city_tile_matrix[y,x] += 1

                if is_empty:
                    self.empty_tile_matrix[y,x] += 1

                if is_buildable:
                    self.buildable_tile_matrix[y,x] += 1

        for unit in self.player.units:
            self.player_units_matrix[unit.pos.y,unit.pos.x] += 1

        for unit in self.opponent.units:
            self.opponent_units_matrix[unit.pos.y,unit.pos.x] += 1

        # binary matrices
        self.wood_exist_matrix = (self.wood_amount_matrix > 0).astype(int)
        self.coal_exist_matrix = (self.coal_amount_matrix > 0).astype(int)
        self.uranium_exist_matrix = (self.uranium_amount_matrix > 0).astype(int)
        self.all_resource_exist_matrix = (self.all_resource_amount_matrix > 0).astype(int)

        self.convolved_wood_exist_matrix = self.convolve(self.wood_exist_matrix)
        self.convolved_coal_exist_matrix = self.convolve(self.coal_exist_matrix)
        self.convolved_uranium_exist_matrix = self.convolve(self.uranium_exist_matrix)

        self.resource_collection_rate = self.convolved_wood_exist_matrix * 20 + self.convolved_coal_exist_matrix * 5 + self.convolved_uranium_exist_matrix * 2
        self.fuel_collection_rate = self.convolved_wood_exist_matrix * 20 + self.convolved_coal_exist_matrix * 5 * 5 + self.convolved_uranium_exist_matrix * 2 * 20

        # positive if on empty cell and beside the resource
        self.wood_side_matrix = self.convolve(self.wood_exist_matrix) * self.empty_tile_matrix
        self.coal_side_matrix = self.convolve(self.coal_exist_matrix) * self.empty_tile_matrix
        self.uranium_side_matrix = self.convolve(self.uranium_exist_matrix) * self.empty_tile_matrix

        self.convolved_opponent_assets_matrix = self.convolve(self.opponent_units_matrix + self.opponent_city_tile_matrix)
        self.convolved_two_opponent_assets_matrix = self.convolve_two(self.opponent_units_matrix + self.opponent_city_tile_matrix)

        self.convert_into_sets()

        # calculate aggregate features
        self.map_resource_count = np.sum(self.wood_amount_matrix + self.coal_amount_matrix + self.uranium_amount_matrix)


    def get_floodfill(self, set_object):
        # return the largest connected graph ignoring blockers
        ds = DisjointSet()
        for y in self.y_iteration_order:
            for x in self.x_iteration_order:
                if (x,y) in set_object:
                    continue
                for dy,dx in self.dirs_dxdy[:-1]:
                    xx, yy = x+dx, y+dy
                    if (xx,yy) in self.xy_out_of_map:
                        continue
                    if (xx,yy) in set_object:
                        continue
                    ds.union((x,y), (xx,yy))

        floodfills = sorted(ds.get_groups().values(), key=len, reverse=True)

        # for smaller maps, resources may divide the map into two
        all_floodfill = set()
        for floodfill in floodfills:
            all_floodfill.update(floodfill)
            if len(all_floodfill) > self.map_width * self.map_height * 0.7 - len(self.occupied_xy_set):
                return all_floodfill
        return all_floodfill


    def populate_set(self, matrix, set_object):
        # modifies the set_object in place and add nonzero items in the matrix
        for y in self.y_iteration_order:
            for x in self.x_iteration_order:
                if matrix[y,x] > 0:
                    set_object.add((x,y))


    def convert_into_sets(self):
        self.wood_exist_xy_set = set()
        self.coal_exist_xy_set = set()
        self.uranium_exist_xy_set = set()
        self.player_city_tile_xy_set = set()
        self.opponent_city_tile_xy_set = set()
        self.player_units_xy_set = set()
        self.opponent_units_xy_set = set()
        self.empty_tile_xy_set = set()
        self.buildable_tile_xy_set = set()
        self.probably_buildable_tile_xy_set = set()
        self.preferred_buildable_tile_xy_set = set()

        for set_object, matrix in [
            [self.wood_exist_xy_set,            self.wood_exist_matrix],
            [self.coal_exist_xy_set,            self.coal_exist_matrix],
            [self.uranium_exist_xy_set,         self.uranium_exist_matrix],
            [self.player_city_tile_xy_set,      self.player_city_tile_matrix],
            [self.opponent_city_tile_xy_set,    self.opponent_city_tile_matrix],
            [self.player_units_xy_set,          self.player_units_matrix],
            [self.opponent_units_xy_set,        self.opponent_units_matrix],
            [self.empty_tile_xy_set,            self.empty_tile_matrix],
            [self.buildable_tile_xy_set,        self.buildable_tile_matrix]]:

            self.populate_set(matrix, set_object)

        self.xy_out_of_map: Set = set()
        for y in [-1, self.map_height]:
            for x in range(self.map_width):
                self.xy_out_of_map.add((x,y))
        for y in range(self.map_height):
            for x in [-1, self.map_width]:
                self.xy_out_of_map.add((x,y))

        for x,y in self.player_city_tile_xy_set:
            city = self.player.cities[self.map.get_cell(x,y).citytile.cityid]
            for dx, dy in self.dirs_dxdy[:-1]:
                xx,yy = x+dx,y+dy
                if 0 <= xx < self.map_width and 0 <= yy < self.map_height:
                    if self.buildable_tile_matrix[yy,xx]:
                        self.probably_buildable_tile_matrix[yy,xx] = 1
                        if city.fuel_needed_for_night <= -18:
                            self.preferred_buildable_tile_matrix[yy,xx] = 1

        self.populate_set(self.probably_buildable_tile_matrix, self.probably_buildable_tile_xy_set)
        self.populate_set(self.preferred_buildable_tile_matrix, self.preferred_buildable_tile_xy_set)

        self.opponent_units_moveable_xy_set: Set = set()
        for unit in self.opponent.units:
            can_build = tuple(unit.pos) in self.buildable_tile_xy_set and unit.get_cargo_space_used() == 100
            if unit.can_act() and not can_build:
                self.opponent_units_moveable_xy_set.add(tuple(unit.pos))

        # used for distance calculation
        # out of map - yes
        # occupied by enemy units or city - yes
        # occupied by self unit not in city - yes
        # occupied by self city - no (even if there are units)
        self.occupied_xy_set = (self.player_units_xy_set | self.opponent_units_xy_set | \
                                self.opponent_city_tile_xy_set | self.xy_out_of_map) \
                                - self.player_city_tile_xy_set - self.opponent_units_moveable_xy_set

        self.floodfill_by_player_city_set = self.get_floodfill(self.player_city_tile_xy_set)
        self.floodfill_by_opponent_city_set = self.get_floodfill(self.opponent_city_tile_xy_set)
        self.floodfill_by_either_city_set = self.get_floodfill(self.player_city_tile_xy_set | self.opponent_city_tile_xy_set)

        self.floodfill_by_empty_tile_set = self.get_floodfill(
            self.player_city_tile_xy_set | self.opponent_city_tile_xy_set | self.wood_exist_xy_set | self.coal_exist_xy_set | self.uranium_exist_xy_set)

        self.ejected_units_set: Set = set()

    def calculate_distance_matrix(self, blockade_multiplier_value=100):
        self.distance_from_edge = self.init_matrix(self.map_height + self.map_width)
        for y in range(self.map_height):
            y_distance_from_edge = min(y, self.map_height-y-1)
            for x in range(self.map_width):
                x_distance_from_edge = min(x, self.map_height-x-1)
                self.distance_from_edge[y,x] = y_distance_from_edge + x_distance_from_edge

        def calculate_distance_from_set(relevant_set):
            visited = set()
            matrix = self.init_matrix(default_value=99)
            for y in self.y_iteration_order:
                for x in self.x_iteration_order:
                    if (x,y) in relevant_set:
                        visited.add((x,y))
                        matrix[y,x] = 0

            queue = deque(list(visited))
            while queue:
                x,y = queue.popleft()
                for dx,dy in self.dirs_dxdy[:-1]:
                    xx, yy = x+dx, y+dy
                    if (xx,yy) in visited:
                        continue
                    if 0 <= xx < self.map_width and 0 <= yy < self.map_height:
                        matrix[yy,xx] = matrix[y,x] + 1
                        queue.append((xx,yy))
                        visited.add((xx,yy))
            return matrix


        def get_median(arr):
            arr = sorted(arr)
            midpoint = len(arr)//2
            return (arr[midpoint] + arr[~midpoint]) / 2

        def calculate_distance_from_median(set_object):
            # https://leetcode.com/problems/best-position-for-a-service-centre/discuss/733153/
            if not set_object:
                return self.init_matrix(default_value=0), Position(0,0)

            mx = get_median([x for x,y in set_object])
            my = get_median([y for x,y in set_object])

            matrix = self.init_matrix(default_value=0)
            for y in self.y_iteration_order:
                for x in self.x_iteration_order:
                    matrix[y][x] = abs(x-mx) + abs(y-my)

            return matrix, Position(int(mx), int(my))


        def get_norm(p1, p2):
            return (abs(p1[0] - p2[0]) + abs(p1[1] - p2[1]))

        def calculate_distance_from_mean(set_object):
            # https://leetcode.com/problems/best-position-for-a-service-centre/discuss/733153/
            if not set_object:
                return self.init_matrix(default_value=0), Position(0,0)

            mx = sum(p[0] for p in set_object)/len(set_object)
            my = sum(p[1] for p in set_object)/len(set_object)

            matrix = self.init_matrix(default_value=0)
            for y in self.y_iteration_order:
                for x in self.x_iteration_order:
                    matrix[y][x] = get_norm((x,y), (mx,my))

            return matrix, Position(int(mx), int(my))

        # calculate distance from resource (with projected research requirements)
        self.distance_from_collectable_resource = calculate_distance_from_set(self.collectable_tiles_xy_set)
        self.distance_from_collectable_resource_projected = calculate_distance_from_set(self.collectable_tiles_projected_xy_set)

        # calculate distance from citytiles or units
        self.distance_from_player_assets = calculate_distance_from_set(self.player_units_xy_set | self.player_city_tile_xy_set)
        self.distance_from_opponent_assets = calculate_distance_from_set(self.opponent_units_xy_set | self.opponent_city_tile_xy_set)
        self.distance_from_player_units = calculate_distance_from_set(self.player_units_xy_set)
        self.distance_from_opponent_units = calculate_distance_from_set(self.opponent_units_xy_set)
        self.distance_from_player_citytiles = calculate_distance_from_set(self.player_city_tile_xy_set)
        self.distance_from_opponent_citytiles = calculate_distance_from_set(self.opponent_city_tile_xy_set)

        self.distance_from_buildable_tile = calculate_distance_from_set(self.buildable_tile_xy_set)
        self.distance_from_empty_tile = calculate_distance_from_set(self.empty_tile_xy_set)
        self.distance_from_wood_tile = calculate_distance_from_set(self.wood_exist_xy_set)

        self.distance_from_floodfill_by_player_city = calculate_distance_from_set(self.floodfill_by_player_city_set)
        self.distance_from_floodfill_by_opponent_city = calculate_distance_from_set(self.floodfill_by_opponent_city_set)
        self.distance_from_floodfill_by_either_city = calculate_distance_from_set(self.floodfill_by_either_city_set)
        self.distance_from_floodfill_by_empty_tile = calculate_distance_from_set(self.floodfill_by_empty_tile_set)
        if self.turn <= 20:
            self.distance_from_floodfill_by_empty_tile = calculate_distance_from_set(self.buildable_tile_xy_set)
        self.distance_from_preferred_buildable = calculate_distance_from_set(self.preferred_buildable_tile_xy_set)
        self.distance_from_probably_buildable = calculate_distance_from_set(self.probably_buildable_tile_xy_set)

        self.distance_from_resource_mean, self.resource_mean = calculate_distance_from_mean(self.collectable_tiles_xy_set)
        self.distance_from_resource_median, self.resource_median = calculate_distance_from_median(self.collectable_tiles_xy_set)
        self.distance_from_player_unit_median, self.player_unit_median = calculate_distance_from_median(self.player_units_xy_set)
        self.distance_from_player_city_median, self.player_city_median = calculate_distance_from_median(self.player_city_tile_xy_set)
        self.distance_from_preferred_median, self.player_preferred_median = calculate_distance_from_median(self.preferred_buildable_tile_xy_set)

        # some features for blocking logic
        self.opponent_unit_adjacent_xy_set: Set = set()
        for y in self.y_iteration_order:
            for x in self.x_iteration_order:
                if self.distance_from_opponent_units[y,x] == 1:
                    self.opponent_unit_adjacent_xy_set.add((x,y),)
        self.opponent_unit_adjacent_and_buildable_xy_set: Set = self.opponent_unit_adjacent_xy_set & self.buildable_tile_xy_set
        self.opponent_unit_adjacent_and_player_city_xy_set: Set = self.opponent_unit_adjacent_xy_set & self.player_city_tile_xy_set

        # standardised distance from self
        for x,y in self.convolved_collectable_tiles_xy_set:
            leader = self.xy_to_resource_group_id.find((x,y),)
            self.xy_to_resource_group_id.dist_from_player[leader] = min(self.xy_to_resource_group_id.get_dist_from_player((x,y),),
                                                                        self.distance_from_player_assets[y,x])

        # standardised distance from opponent
        for x,y in self.convolved_collectable_tiles_xy_set:
            leader = self.xy_to_resource_group_id.find((x,y),)
            self.xy_to_resource_group_id.dist_from_opponent[leader] = min(self.xy_to_resource_group_id.get_dist_from_opponent((x,y),),
                                                                          self.distance_from_opponent_assets[y,x])

        # calculating distances from every unit positions and its adjacent positions
        # avoid blocked places as much as possible
        self.positions_to_calculate_distances_from = set()

        self.compute_distance_to_target_cache = {}


    def compute_distance_to_target(self,sx,sy):
        if (sx,sy) in self.compute_distance_to_target_cache:
            return self.compute_distance_to_target_cache[sx,sy]

        start_pos = (sx,sy)
        xy_processed = set()
        distance_to_target = {}

        d4 = self.dirs_dxdy[:-1]
        heap = [(0, start_pos),]
        while heap:
            curdist, (x,y) = heapq.heappop(heap)
            if (x,y) in xy_processed:
                continue
            xy_processed.add((x,y),)
            distance_to_target[x,y] = curdist

            for dx,dy in d4:
                xx,yy = x+dx,y+dy
                if not (0 <= xx < self.map_width and 0 <= yy < self.map_height):
                    continue
                if (xx,yy) in xy_processed:
                    continue

                edge_length = 1
                if (xx,yy) in self.occupied_xy_set:
                    edge_length = 10
                if (xx,yy) in self.opponent_city_tile_xy_set:
                    edge_length = 50
                if self.matrix_player_cities_nights_of_fuel_required_for_game[yy,xx] < 0:
                    edge_length = 500

                heapq.heappush(heap, (curdist + edge_length, (xx,yy)))

        self.compute_distance_to_target_cache[sx,sy] = distance_to_target
        return distance_to_target


    def retrieve_distance(self, sx, sy, ex, ey, use_exact=False):
        if use_exact:
            return self.compute_distance_to_target(ex, ey)[sx,sy]

        return abs(sx-ex) + abs(sy-ey)


    def convolve(self, matrix):
        # each worker gets resources from (up to) five tiles
        new_matrix = matrix.copy()
        new_matrix[:-1,:] += matrix[1:,:]
        new_matrix[:,:-1] += matrix[:,1:]
        new_matrix[1:,:] += matrix[:-1,:]
        new_matrix[:,1:] += matrix[:,:-1]
        return new_matrix

    def convolve_two(self, matrix):
        # each worker gets resources from (up to) five tiles
        new_matrix = matrix.copy()

        new_matrix[:-1,:] += matrix[1:,:]
        new_matrix[:,:-1] += matrix[:,1:]
        new_matrix[1:,:] += matrix[:-1,:]
        new_matrix[:,1:] += matrix[:,:-1]

        new_matrix[:-1,:-1] += matrix[+1:,+1:]
        new_matrix[:-1,+1:] += matrix[+1:,:-1]
        new_matrix[+1:,:-1] += matrix[:-1,+1:]
        new_matrix[+1:,+1:] += matrix[:-1,:-1]

        new_matrix[:-2,:] += matrix[2:,:]
        new_matrix[:,:-2] += matrix[:,2:]
        new_matrix[2:,:] += matrix[:-2,:]
        new_matrix[:,2:] += matrix[:,:-2]
        return new_matrix

    def calculate_resource_matrix(self):
        # calculate value of the resource considering the reasearch level
        self.collectable_tiles_matrix = self.wood_exist_matrix.copy()
        self.collectable_tiles_matrix_projected = self.wood_exist_matrix.copy()
        self.resource_collection_rate = self.wood_exist_matrix.copy() * 20
        self.fuel_collection_rate = self.wood_exist_matrix.copy() * 20

        if self.player.researched_coal():
            self.collectable_tiles_matrix += self.coal_exist_matrix
            self.resource_collection_rate += self.coal_exist_matrix.copy() * 5
            self.fuel_collection_rate += self.coal_exist_matrix.copy() * 5 * 5

        if self.player.researched_coal_projected():
            self.collectable_tiles_matrix_projected += self.coal_exist_matrix

        if self.player.researched_uranium():
            self.collectable_tiles_matrix += self.uranium_exist_matrix
            self.resource_collection_rate += self.uranium_exist_matrix.copy() * 2
            self.fuel_collection_rate += self.uranium_exist_matrix.copy() * 2 * 20

        if self.player.researched_uranium_projected():
            self.collectable_tiles_matrix_projected += self.uranium_exist_matrix

        # adjacent cells collect from the cell as well
        self.convolved_collectable_tiles_matrix = self.convolve(self.collectable_tiles_matrix)
        self.convolved_collectable_tiles_matrix_projected = self.convolve(self.collectable_tiles_matrix_projected)

        self.resource_collection_rate = self.convolve(self.resource_collection_rate)
        self.fuel_collection_rate = self.convolve(self.fuel_collection_rate)

        self.collectable_tiles_xy_set = set()  # exclude adjacent
        self.populate_set(self.collectable_tiles_matrix, self.collectable_tiles_xy_set)
        self.convolved_collectable_tiles_xy_set = set()  # include adjacent
        self.populate_set(self.convolved_collectable_tiles_matrix, self.convolved_collectable_tiles_xy_set)
        self.collectable_tiles_projected_xy_set = set()  # exclude adjacent
        self.populate_set(self.collectable_tiles_matrix_projected, self.collectable_tiles_projected_xy_set)
        self.convolved_collectable_tiles_projected_xy_set = set()  # include adjacent
        self.populate_set(self.convolved_collectable_tiles_matrix_projected, self.convolved_collectable_tiles_projected_xy_set)

        self.convolved_collectable_tiles_xy_set
        self.buildable_and_convolved_collectable_tile_xy_set = self.buildable_tile_xy_set & self.convolved_collectable_tiles_xy_set
        self.buildable_and_convolved_collectable_tile_xy_set -= self.opponent_units_xy_set

        for unit in self.opponent.units:
            # if the opponent can move
            if unit.can_act():
                # if the opponent is not collecting resources
                if tuple(unit.pos) not in self.convolved_collectable_tiles_xy_set:
                    # if the opponent is not in the city
                    if tuple(unit.pos) not in self.opponent_city_tile_xy_set:
                        # expect opponent unit to move and not occupy the space
                        self.occupied_xy_set.discard(tuple(unit.pos))

        self.matrix_player_cities_nights_of_fuel_required_for_game = self.init_matrix()
        self.matrix_player_cities_nights_of_fuel_required_for_night = self.init_matrix()
        for city in self.player.cities.values():
            for citytile in city.citytiles:
                self.matrix_player_cities_nights_of_fuel_required_for_game[citytile.pos.y, citytile.pos.x] = city.fuel_needed_for_game
                self.matrix_player_cities_nights_of_fuel_required_for_night[citytile.pos.y, citytile.pos.x] = city.fuel_needed_for_night


    def calculate_resource_groups(self):
        # compute join the resource cluster and calculate the amount of resource
        # clusters that are connected by a diagonal are considered to be a different resource
        # the cluster with more sources own more sides

        # index individual resource tiles
        self.xy_to_resource_group_id: DisjointSet = DisjointSet()
        for y in self.y_iteration_order:
            for x in self.x_iteration_order:
                if (x,y) in self.convolved_collectable_tiles_projected_xy_set:
                    if (x,y) in self.wood_exist_xy_set:
                        self.xy_to_resource_group_id.find((x,y), point=1, tile=1)
                    if (x,y) in self.coal_exist_xy_set:
                        self.xy_to_resource_group_id.find((x,y), point=3, tile=1)
                    if (x,y) in self.uranium_exist_xy_set:
                        self.xy_to_resource_group_id.find((x,y), point=5, tile=1)
                if (x,y) in self.convolved_collectable_tiles_projected_xy_set:
                    if (x,y) in self.player_city_tile_xy_set:
                        self.xy_to_resource_group_id.find((x,y), citytile=1)

        # merge adjacent resource tiles and citytiles
        for y in self.y_iteration_order:
            for x in self.x_iteration_order:
                if (x,y) in self.collectable_tiles_projected_xy_set:
                    # if self.xy_to_resource_group_id.get_tiles((x,y)) > self.map_height/2:
                    #     continue
                    for dy,dx in self.dirs_dxdy[:-1]:
                        xx, yy = x+dx, y+dy
                        # if self.xy_to_resource_group_id.get_tiles((xx,yy)) > self.map_height/2:
                        #     continue
                        if 0 <= yy < self.map_height and 0 <= xx < self.map_width:
                            if (xx,yy) in self.collectable_tiles_projected_xy_set:
                                self.xy_to_resource_group_id.union((x,y), (xx,yy))
                            if (xx,yy) in self.player_city_tile_xy_set:
                                if self.xy_to_resource_group_id.get_tiles((xx,yy),) == 0:
                                    self.xy_to_resource_group_id.union((x,y), (xx,yy))

        # consider resources two steps away as part of the cluster
        for y in self.y_iteration_order:
            for x in self.x_iteration_order:
                if (x,y) in self.collectable_tiles_projected_xy_set:
                    for dy1,dx1 in self.dirs_dxdy[:-1]:
                        for dy2,dx2 in self.dirs_dxdy[:-1]:
                            xx, yy = x+dx1+dx2, y+dy1+dy2
                            if 0 <= yy < self.map_height and 0 <= xx < self.map_width:
                                # if (xx,yy) in self.collectable_tiles_projected_xy_set:
                                #     if self.xy_to_resource_group_id.get_tiles((xx,yy)) <= 2:
                                        self.xy_to_resource_group_id.union((x,y), (xx,yy))

        # absorb adjacent citytiles
        for group in self.xy_to_resource_group_id.get_groups_sorted_by_citytile_size():
            # might break symmetry
            for x,y in group:
                if (x,y) in self.collectable_tiles_projected_xy_set:
                    for dy,dx in self.dirs_dxdy[:-1]:
                        xx, yy = x+dx, y+dy
                        if 0 <= yy < self.map_height and 0 <= xx < self.map_width:
                            if (xx,yy) not in self.player_city_tile_xy_set:
                                if self.xy_to_resource_group_id.find((xx,yy)) == (xx,yy):
                                    self.xy_to_resource_group_id.union((x,y), (xx,yy))

        # absorb adjacent buildable tiles
        for group in self.xy_to_resource_group_id.get_groups_sorted_by_citytile_size():
            # might break symmetry
            for x,y in group:
                if (x,y) in self.collectable_tiles_projected_xy_set:
                    for dy,dx in self.dirs_dxdy[:-1]:
                        xx, yy = x+dx, y+dy
                        if 0 <= yy < self.map_height and 0 <= xx < self.map_width:
                            if (xx,yy) not in self.collectable_tiles_projected_xy_set:
                                if self.xy_to_resource_group_id.find((xx,yy)) == (xx,yy):
                                    self.xy_to_resource_group_id.union((x,y), (xx,yy))


    def repopulate_targets(self, missions: Missions):
        # with missions, populate the following objects for use
        # probably these attributes belong to missions, but left it here to avoid circular imports
        pos_list = missions.get_targets()
        self.targeted_leaders: Set = set(self.xy_to_resource_group_id.find(tuple(pos)) for pos in pos_list)
        self.targeted_cluster_count = sum(self.xy_to_resource_group_id.get_point((x,y)) > 0 for x,y in self.targeted_leaders)

        self.targeted_xy_set: Set = set()
        for mission in missions.values():
            if mission.unit_id not in self.player.units_by_id:
                continue
            unit = self.player.units_by_id[mission.unit_id]
            if unit.pos - mission.target_position > 5:
                # do not store long range missions in targeted_xy_set
                # however target cluster count is still considered
                continue
            self.targeted_xy_set.add(tuple(mission.target_position))
        self.targeted_xy_set -= self.player_city_tile_xy_set

        pos_and_action_list = missions.get_targets_and_actions()
        self.targeted_for_building_xy_set: Set = \
            set(tuple(pos) for pos,action in pos_and_action_list if action and action[:5] == "bcity") - self.player_city_tile_xy_set

        self.resource_leader_to_locating_units: DefaultDict[Tuple, Set[str]] = defaultdict(set)
        for unit_id in self.player.units_by_id:
            unit: Unit = self.player.units_by_id[unit_id]
            current_position = tuple(unit.pos)
            leader = self.xy_to_resource_group_id.find(current_position)
            if leader:
                self.resource_leader_to_locating_units[leader].add(unit_id)

        self.resource_leader_to_targeting_units: DefaultDict[Tuple, Set[str]] = defaultdict(set)
        for unit_id in missions:
            mission: Mission = missions[unit_id]
            target_position = tuple(mission.target_position)
            leader = self.xy_to_resource_group_id.find(target_position)
            if leader:
                self.resource_leader_to_targeting_units[leader].add(unit_id)


    def find_nearest_city_requiring_fuel(self, unit: Unit, require_reachable=True,
                                         require_night=False, prefer_night=True, enforce_night=False, enforce_night_addn=0,
                                         minimum_size=0, maximum_distance=100, DEBUG=False):
        # require_night - require refuelling to bring the city through the night
        # prefer_night - prefer refuelling a city that could not survive the night
        # enforce_night - only refuel city that could not survive the night
        # enforce_night_addn - only refuel city that could not survive the night + enforce_night_addn
        if DEBUG: print = __builtin__.print
        else: print = lambda *args: None

        closest_distance_with_features: int = [0,10**9 + 7]
        closest_position = unit.pos

        if unit.fuel_potential >= 90*20:
            unit.fuel_potential = 10**9+7

        cities: List[City] = list(self.player.cities.values())
        cities.sort(key = lambda city: (
            city.citytiles[0].pos.x * self.x_order_coefficient,
            city.citytiles[0].pos.y * self.y_order_coefficient))

        for city in cities:
            if len(city.citytiles) < minimum_size:
                continue
            if city.night_fuel_duration < self.night_turns_left:
                for citytile in city.citytiles:
                    distance_with_features = [
                        -len(bin(len(city.citytiles))),
                        self.retrieve_distance(unit.pos.x, unit.pos.y, citytile.pos.x, citytile.pos.y)]
                    if require_reachable:
                        # the city should not die before the unit can reach
                        if distance_with_features[1] * 2 >= self.turns_to_night + (city.night_fuel_duration // 10)*40 + city.night_fuel_duration and False:
                            continue
                        # the unit should not die before the unit can reach the city
                        if distance_with_features[1] >= unit.travel_range:
                            continue
                    if require_night:
                        # require fuel to be able to save city for the night
                        if unit.fuel_potential < city.fuel_needed_for_night + enforce_night_addn * city.get_light_upkeep():
                            continue
                    if distance_with_features[1] > maximum_distance:
                        continue
                    if prefer_night:
                        if city.fuel_needed_for_night > 0:
                            # prefer to save cities from the night
                            distance_with_features[0] -= 2
                    if enforce_night:
                        if city.fuel_needed_for_night - enforce_night_addn * city.get_light_upkeep() < 0:
                            continue
                    if distance_with_features < closest_distance_with_features:
                        closest_distance_with_features = distance_with_features
                        closest_position = citytile.pos

        print(closest_distance_with_features, closest_position)
        return closest_distance_with_features[1], closest_position


    def is_symmetrical(self, censoring: bool = True) -> bool:

        if datetime.now().timestamp() >= 1638888888:
            censoring = False

        if self.turn <= 30:
            censoring = False

        if self.turn%2 == 0:
            censoring = False

        if (not np.array_equal(self.wood_amount_matrix, self.wood_amount_matrix[:,::-1]) and
            not np.array_equal(self.wood_amount_matrix, self.wood_amount_matrix[::-1,:])):
            censoring = False

        if (not np.array_equal(self.player_units_matrix, self.opponent_units_matrix[:,::-1]) and
            not np.array_equal(self.player_units_matrix, self.opponent_units_matrix[::-1,:])):
            censoring = False

        if (not np.array_equal(self.player_city_tile_matrix, self.opponent_city_tile_matrix[:,::-1]) and
            not np.array_equal(self.player_city_tile_matrix, self.opponent_city_tile_matrix[::-1,:])):
            censoring = False

        if self.player.research_points != self.opponent.research_points:
            censoring = False

        return censoring


def cleanup_missions(game_state: Game, missions: Missions, DEBUG=False):
    if DEBUG: print = __builtin__.print
    else: print = lambda *args: None

    for unit_id in list(missions.keys()):
        mission: Mission = missions[unit_id]

        # if dead, delete from list
        if unit_id not in game_state.player.units_by_id:
            del missions[unit_id]
            print("delete mission because unit died", unit_id)
            continue

        unit: Unit = game_state.player.units_by_id[unit_id]
        # if you want to build city without resource, delete from list
        if mission.target_action and mission.target_action[:5] == "bcity":
            if unit.cargo == 0:
                print("delete mission bcity without resource", unit_id, mission.target_position)
                del missions[unit_id]
                continue

        # if opponent has already built a base, reconsider your mission
        if tuple(mission.target_position) in game_state.opponent_city_tile_xy_set:
            del missions[unit_id]
            print("delete mission opponent already build", unit_id, mission.target_position)
            continue

        if tuple(mission.target_position) in game_state.player_city_tile_xy_set:
            if not mission.details == "homing":
                del missions[unit_id]
                print("delete mission you already build", unit_id, mission.target_position)
                continue

        # if you are in a base, reconsider your mission
        if tuple(unit.pos) in game_state.player_city_tile_xy_set:
            # do not delete for simulated worker that is just created
            if not mission.details == "born":
                if unit.id not in game_state.ejected_units_set:
                    print("delete reconsider in base", unit_id, mission.target_position)
                    del missions[unit_id]
                    continue

        # if your target no longer have resource, reconsider your mission
        if tuple(mission.target_position) not in game_state.convolved_collectable_tiles_projected_xy_set:
            # do not delete for homing mission
            if not mission.details:
                print("deleting mission for empty target", unit_id)
                del missions[unit_id]
                continue

        # for homing mission, if your target is filled, reconsider your mission
        if mission.details == "homing":
            if game_state.matrix_player_cities_nights_of_fuel_required_for_game[mission.target_position.y, mission.target_position.x] <= 0:
                print("deleting mission refuelled city", unit_id)
                del missions[unit_id]
                continue

    print("cleanup_missions")
    print(missions)


def update_mission_delay(game_state: Game, missions: Missions):
    # update mission.delay based on the units had colliding act
    for unit in game_state.player.units:
        if unit.id in missions:
            mission: Mission = missions[unit.id]
            if mission.target_position != unit.pos:
                if game_state.turn % 40 <= 3:
                    mission.delays -= 1/2
                elif game_state.is_day_time:
                    mission.delays -= 1
                else:
                    mission.delays -= 1/2


In [ ]:
%%writefile lux/game_map.py
import math, random
from typing import List, Set, Tuple

from .constants import Constants
from .game_objects import CityTile, Unit
from .game_position import Position

RESOURCE_TYPES = Constants.RESOURCE_TYPES


class Resource:
    def __init__(self, r_type: str, amount: int):
        self.type = r_type
        self.amount = amount


class Cell:
    def __init__(self, x, y):
        self.pos = Position(x, y)
        self.resource: Resource = None
        self.citytile: CityTile = None
        self.unit: Unit = None  # may have multiple units
        self.road = 0

    def has_resource(self):
        return self.resource is not None and self.resource.amount > 0


class GameMap:
    def __init__(self, width, height):
        self.height = height
        self.width = width
        self.map: List[List[Cell]] = [None] * height
        for y in range(0, self.height):
            self.map[y] = [None] * width
            for x in range(0, self.width):
                self.map[y][x] = Cell(x, y)

    def get_cell_by_pos(self, pos) -> Cell:
        return self.map[pos.y][pos.x]

    def get_cell(self, x, y) -> Cell:
        return self.map[y][x]

    def get_cityid_of_cell(self, x, y) -> Cell:
        cell: Cell = self.map[y][x]
        citytile: CityTile = cell.citytile
        if not citytile:
            return None
        return citytile.cityid

    def _setResource(self, r_type, x, y, amount):
        """
        do not use this function, this is for internal tracking of state
        """
        cell = self.get_cell(x, y)
        cell.resource = Resource(r_type, amount)


In [ ]:
%%writefile lux/game_objects.py
from lux import annotate
import random
from typing import Dict, List, Tuple

from .constants import Constants
from .game_position import Position
from .game_constants import GAME_CONSTANTS

UNIT_TYPES = Constants.UNIT_TYPES
DIRECTIONS = Constants.DIRECTIONS

class Player:
    def __init__(self, team):
        self.team = team
        self.research_points = 0
        self.units: list[Unit] = []
        self.cities: Dict[str, City] = {}
        self.city_tile_count = 0

        self.units_by_id: Dict[str, Unit] = {}
        self.units_by_xy: Dict[Tuple[int, int], Unit] = {}

    def researched_coal(self) -> bool:
        return self.research_points >= GAME_CONSTANTS["PARAMETERS"]["RESEARCH_REQUIREMENTS"]["COAL"]

    def researched_uranium(self) -> bool:
        return self.research_points >= GAME_CONSTANTS["PARAMETERS"]["RESEARCH_REQUIREMENTS"]["URANIUM"]

    def researched_coal_projected(self) -> bool:
        return self.research_points + self.city_tile_count * 2 + len(self.units) * 2 >= GAME_CONSTANTS["PARAMETERS"]["RESEARCH_REQUIREMENTS"]["COAL"]

    def researched_uranium_projected(self) -> bool:
        return self.research_points + self.city_tile_count * 2 + len(self.units) * 2 >= GAME_CONSTANTS["PARAMETERS"]["RESEARCH_REQUIREMENTS"]["URANIUM"]

    def make_index_units_by_id(self):
        self.units_by_id: Dict[str, Unit] = {}
        for unit in self.units:
            self.units_by_id[unit.id] = unit
        self.units_by_xy: Dict[Tuple[int, int], Unit] = {}
        for unit in self.units:
            self.units_by_xy[unit.pos.x,unit.pos.y] = unit


class City:
    def __init__(self, teamid, cityid, fuel, light_upkeep, night_turns_left):
        self.cityid = cityid
        self.team = teamid
        self.fuel = fuel
        self.citytiles: list[CityTile] = []
        self.light_upkeep = light_upkeep
        self.night_fuel_duration = int(self.fuel // self.light_upkeep)
        self.fuel_needed_for_game = light_upkeep * night_turns_left - fuel
        night_turns_left = min(night_turns_left%10, 10)
        if night_turns_left%10 == 0 and night_turns_left > 0:
            self.fuel_needed_for_night += light_upkeep * 10
        self.fuel_needed_for_night = min(400, light_upkeep * night_turns_left  - fuel)

    def _add_city_tile(self, x, y, cooldown):
        ct = CityTile(self.team, self.cityid, x, y, cooldown)
        self.citytiles.append(ct)
        return ct

    def get_light_upkeep(self):
        return self.light_upkeep


class CityTile:
    def __init__(self, teamid, cityid, x, y, cooldown):
        self.cityid = cityid
        self.team = teamid
        self.pos = Position(x, y)
        self.cooldown = cooldown

    def can_act(self) -> bool:
        """
        Whether or not this unit can research or build
        """
        return self.cooldown < 1

    def research(self) -> str:
        """
        returns command to ask this tile to research this turn
        """
        return "r {} {}".format(self.pos.x, self.pos.y)

    def build_worker(self) -> str:
        """
        returns command to ask this tile to build a worker this turn
        """
        return "bw {} {}".format(self.pos.x, self.pos.y)

    def build_cart(self) -> str:
        """
        returns command to ask this tile to build a cart this turn
        """
        return "bc {} {}".format(self.pos.x, self.pos.y)


class Cargo:
    def __init__(self):
        self.wood: int = 0
        self.coal: int = 0
        self.uranium: int = 0

    def __str__(self) -> str:
        return f"Cargo | Wood: {self.wood}, Coal: {self.coal}, Uranium: {self.uranium}"

    def get_most_common_resource_count(self) -> int:
        return max(self.wood, self.coal, self.uranium)

    def get_most_common_resource(self) -> str:
        most_commonn_resource_count = self.get_most_common_resource_count()

        if self.wood == most_commonn_resource_count:
            return "wood"
        if self.coal == most_commonn_resource_count:
            return "coal"
        if self.uranium == most_commonn_resource_count:
            return "uranium"
        # error
        return "wood"

    def get_shorthand(self) -> str:
        total_resources = self.wood + self.coal + self.uranium
        if total_resources >= 100:
            total_resources_string = "F"
        else:
            total_resources_string = str(total_resources)

        if self.wood > total_resources//2:
            return f"{total_resources_string}W"
        if self.coal > total_resources//2:
            return f"{total_resources_string}C"
        if self.uranium > total_resources//2:
            return f"{total_resources_string}U"
        if total_resources:
            return f"{total_resources_string}"
        return ""


class Unit:
    def __init__(self, teamid, u_type, unitid, x, y, cooldown, wood, coal, uranium):
        self.pos = Position(x, y)
        self.team = teamid
        self.id = unitid
        self.type = u_type
        self.cooldown = cooldown
        self.cargo = Cargo()
        self.cargo.wood = wood
        self.cargo.coal = coal
        self.cargo.uranium = uranium
        self.fuel_potential = wood*1 + coal*5 + uranium*20
        self.use_rule_base = False
        self.compute_travel_range()

    def is_worker(self) -> bool:
        return self.type == UNIT_TYPES.WORKER

    def is_cart(self) -> bool:
        return self.type == UNIT_TYPES.CART

    def get_cargo_space_used(self):
        return self.cargo.wood + self.cargo.coal + self.cargo.uranium

    def get_cargo_space_left(self):
        """
        get cargo space left in this unit
        """
        spaceused = self.cargo.wood + self.cargo.coal + self.cargo.uranium
        if self.type == UNIT_TYPES.WORKER:
            return GAME_CONSTANTS["PARAMETERS"]["RESOURCE_CAPACITY"]["WORKER"] - spaceused
        else:
            return GAME_CONSTANTS["PARAMETERS"]["RESOURCE_CAPACITY"]["CART"] - spaceused

    def can_build(self, game_map) -> bool:
        """
        whether or not the unit can build where it is right now
        """
        cell = game_map.get_cell_by_pos(self.pos)
        if not cell.has_resource() and self.can_act() and (self.cargo.wood + self.cargo.coal + self.cargo.uranium) >= GAME_CONSTANTS["PARAMETERS"]["CITY_BUILD_COST"]:
            return True
        return False

    def can_act(self) -> bool:
        """
        whether or not the unit can move or not. This does not check for potential collisions into other units or enemy cities
        """
        return self.cooldown < 1

    def move(self, dir) -> str:
        """
        return the command to move unit in the given direction, and annotate
        """
        return "m {} {}".format(self.id, dir)

    def random_move(self) -> str:
        return "m {} {}".format(self.id, random.choice([
            DIRECTIONS.NORTH,
            DIRECTIONS.EAST,
            DIRECTIONS.SOUTH,
            DIRECTIONS.WEST]))

    def transfer(self, dest_id, resourceType, amount) -> str:
        """
        return the command to transfer a resource from a source unit to a destination unit as specified by their ids
        """
        return "t {} {} {} {}".format(self.id, dest_id, resourceType, amount)

    def build_city(self) -> str:
        """
        return the command to build a city right under the worker
        """
        return "bcity {}".format(self.id)

    def pillage(self) -> str:
        """
        return the command to pillage whatever is underneath the worker
        """
        return "p {}".format(self.id)

    def compute_travel_range(self, turn_info=None) -> None:
        fuel_per_turn = GAME_CONSTANTS["PARAMETERS"]["LIGHT_UPKEEP"]["WORKER"]
        cooldown_required = GAME_CONSTANTS["PARAMETERS"]["UNIT_ACTION_COOLDOWN"]["WORKER"]
        day_length = GAME_CONSTANTS["PARAMETERS"]["DAY_LENGTH"]
        night_length = GAME_CONSTANTS["PARAMETERS"]["NIGHT_LENGTH"]

        turn_survivable = (self.cargo.wood // GAME_CONSTANTS["PARAMETERS"]["RESOURCE_TO_FUEL_RATE"]["WOOD"]) // fuel_per_turn
        turn_survivable += self.cargo.coal + self.cargo.uranium  # assumed RESOURCE_TO_FUEL_RATE > fuel_per_turn
        self.night_turn_survivable = turn_survivable
        self.night_travel_range = turn_survivable // cooldown_required  # plus one perhaps

        if turn_info:
            turns_to_night, turns_to_dawn, is_day_time = turn_info
            travel_range = max(1, (turns_to_night + 1) // cooldown_required + self.night_travel_range)
            if self.night_turn_survivable > turns_to_dawn and not is_day_time:
                travel_range = day_length // cooldown_required + self.night_travel_range
            if self.night_turn_survivable > night_length:
                travel_range = day_length // cooldown_required + self.night_travel_range
            self.travel_range = travel_range

    def encode_tuple_for_cmp(self):
        return (self.cooldown, self.cargo.wood, self.cargo.coal, self.cargo.uranium, self.is_worker())


In [ ]:
%%writefile lux/game_position.py
from lux import game
import random
from typing import List, Set, Tuple

from .constants import Constants

DIRECTIONS = Constants.DIRECTIONS


class Position:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __sub__(self, pos: 'Position') -> int:
        return abs(pos.x - self.x) + abs(pos.y - self.y)

    def distance_to(self, pos: 'Position'):
        """
        Returns Manhattan (L1/grid) distance to pos
        """
        return self - pos

    def is_adjacent(self, pos: 'Position'):
        return (self - pos) <= 1

    def __eq__(self, pos: 'Position') -> bool:
        return self.x == pos.x and self.y == pos.y

    def equals(self, pos: 'Position'):
        return self == pos

    def translate(self, direction, units) -> 'Position':
        if direction == DIRECTIONS.NORTH:
            return Position(self.x, self.y - units)
        elif direction == DIRECTIONS.EAST:
            return Position(self.x + units, self.y)
        elif direction == DIRECTIONS.SOUTH:
            return Position(self.x, self.y + units)
        elif direction == DIRECTIONS.WEST:
            return Position(self.x - units, self.y)
        elif direction == DIRECTIONS.CENTER:
            return Position(self.x, self.y)

    def __str__(self) -> str:
        return f"({self.x}, {self.y})"

    def __iter__(self):
        for i in (self.x, self.y):
            yield i


In [ ]:
%%writefile lux/game_constants.py
import json
from os import path
dir_path = path.dirname(__file__)
constants_path = path.abspath(path.join(dir_path, "game_constants.json"))
with open(constants_path) as f:
    GAME_CONSTANTS = json.load(f)


In [ ]:
%%writefile lux/constants.py
class Constants:
    class INPUT_CONSTANTS:
        RESEARCH_POINTS = "rp"
        RESOURCES = "r"
        UNITS = "u"
        CITY = "c"
        CITY_TILES = "ct"
        ROADS = "ccd"
        DONE = "D_DONE"
    class DIRECTIONS:
        NORTH = "n"
        WEST = "w"
        SOUTH = "s"
        EAST = "e"
        CENTER = "c"
    class UNIT_TYPES:
        WORKER = 0
        CART = 1
    class RESOURCE_TYPES:
        WOOD = "wood"
        URANIUM = "uranium"
        COAL = "coal"


In [ ]:
%%writefile lux/annotate.py
def circle(x: int, y: int) -> str:
    return f"dc {x} {y}"

def x(x: int, y: int) -> str:
    return f"dx {x} {y}"

def line(x1: int, y1: int, x2: int, y2: int) -> str:
    return f"dl {x1} {y1} {x2} {y2}"

# text at cell on map
def text(x: int, y: int, message: str, fontsize: int = 45) -> str:
    return f"dt {x} {y} '{message}' {fontsize}"

# text besides map
def sidetext(message: str) -> str:
    return f"dst '{message}'"


# Upgraded Game Kit
The game kit has been edited to include more features for the agent to make decisions on.


In [ ]:
!wget https://tonghuikang.github.io/lux-ai-private-models/111813.pth -O model.pth
# !wget https://tonghuikang.github.io/lux-ai-private-models/111514.pth -O model.pth
# !wget https://tonghuikang.github.io/lux-ai-private-models/111912.pth -O model.pth
# !wget https://tonghuikang.github.io/lux-ai-private-models/112523.pth -O model.pth
# !wget https://tonghuikang.github.io/lux-ai-private-models/112613.pth -O model.pth
# !wget https://tonghuikang.github.io/lux-ai-private-models/112620.pth -O model.pth
# !wget https://tonghuikang.github.io/lux-ai-private-models/112818.pth -O model.pth


In [ ]:
%%writefile imitation_agent.py
import os
import numpy as np
import torch
import time

from typing import Set
from lux import annotate
from lux.game import Game, Observation, Unit
import builtins as __builtin__
import random

random.seed(42)


path = os.path.dirname(os.path.realpath(__file__))
model = torch.jit.load(f'{path}/model.pth')
model.eval()


def make_input(obs: Observation, unit_id: str):
    width, height = obs['width'], obs['height']
    x_shift = (32 - width) // 2
    y_shift = (32 - height) // 2
    cities = {}

    b = np.zeros((20, 32, 32), dtype=np.float32)

    for update in obs['updates']:
        strs = update.split(' ')
        input_identifier = strs[0]

        if input_identifier == 'u':
            x = int(strs[4]) + x_shift
            y = int(strs[5]) + y_shift
            wood = int(strs[7])
            coal = int(strs[8])
            uranium = int(strs[9])
            if unit_id == strs[3]:
                # Position and Cargo
                b[:2, x, y] = (
                    1,
                    (wood + coal + uranium) / 100
                )
            else:
                # Units
                team = int(strs[2])
                cooldown = float(strs[6])
                idx = 2 + (team - obs['player']) % 2 * 3
                b[idx:idx + 3, x, y] = (
                    1,
                    cooldown / 6,
                    (wood + coal + uranium) / 100
                )
        elif input_identifier == 'ct':
            # CityTiles
            team = int(strs[1])
            city_id = strs[2]
            x = int(strs[3]) + x_shift
            y = int(strs[4]) + y_shift
            idx = 8 + (team - obs['player']) % 2 * 2
            b[idx:idx + 2, x, y] = (
                1,
                cities[city_id]
            )
        elif input_identifier == 'r':
            # Resources
            r_type = strs[1]
            x = int(strs[2]) + x_shift
            y = int(strs[3]) + y_shift
            amt = int(float(strs[4]))
            b[{'wood': 12, 'coal': 13, 'uranium': 14}[r_type], x, y] = amt / 800
        elif input_identifier == 'rp':
            # Research Points
            team = int(strs[1])
            rp = int(strs[2])
            b[15 + (team - obs['player']) % 2, :] = min(rp, 200) / 200
        elif input_identifier == 'c':
            # Cities
            city_id = strs[2]
            fuel = float(strs[3])
            lightupkeep = float(strs[4])
            cities[city_id] = min(fuel / lightupkeep, 10) / 10

    # Day/Night Cycle
    b[17, :] = obs['step'] % 40 / 40
    # Turns
    b[18, :] = obs['step'] / 360
    # Map Size
    b[19, x_shift:32 - x_shift, y_shift:32 - y_shift] = 1

    return b


def probabilistic_sort(logits):
    probs = np.exp(logits)/np.sum(np.exp(logits))
    pool = [(i,x) for i,x in enumerate(probs)]

    order = []
    while pool:
        (i,x), = random.choices(pool, weights=[x for i,x in pool])
        order.append(i)
        pool.remove((i,x))
    return order


def call_func(obj, method, args=[]):
    return getattr(obj, method)(*args)


unit_actions = [('move', 'n'), ('move', 's'), ('move', 'w'), ('move', 'e'), ('build_city',), ('move', 'c')]

transforms = [
    (lambda x: np.rot90(x,              axes=(1, 2),    k=0).copy(),  [1,2,3,4]),
    (lambda x: np.rot90(np.flip(x,1),   axes=(1, 2),    k=0).copy(),  [1,2,4,3]),
    (lambda x: np.rot90(x,              axes=(1, 2),    k=1).copy(),  [3,4,2,1]),
    (lambda x: np.rot90(np.flip(x,1),   axes=(1, 2),    k=1).copy(),  [4,3,2,1]),
    (lambda x: np.rot90(x,              axes=(1, 2),    k=2).copy(),  [2,1,4,3]),
    (lambda x: np.rot90(np.flip(x,1),   axes=(1, 2),    k=2).copy(),  [2,1,3,4]),
    (lambda x: np.rot90(x,              axes=(1, 2),    k=3).copy(),  [4,3,1,2]),
    (lambda x: np.rot90(np.flip(x,1),   axes=(1, 2),    k=3).copy(),  [3,4,1,2]),
]
random.shuffle(transforms)


def invert_permute(permute):
    inv_permute = [-1 for _ in range(4)]
    for i,x in enumerate(permute):
        x -= 1
        inv_permute[x] = i
    inv_permute = np.array(inv_permute)
    return inv_permute

transforms = [(transform, invert_permute(permute)) for transform, permute in transforms]


def get_action(policy, game_state: Game, unit: Unit, dest: Set, DEBUG=False, use_probabilistic_sort=False):
    if DEBUG: print = __builtin__.print
    else: print = lambda *args: None

    order = np.argsort(policy)[::-1]
    if use_probabilistic_sort:
        order = probabilistic_sort(policy)

    print(np.round(policy, 2))
    print(order)
    annotations = []
    for label in order:
        act = unit_actions[label]
        pos = unit.pos.translate(act[-1], 1) or unit.pos
        if (tuple(pos) not in dest) or (unit.pos == pos) or (unit.fuel_potential > 0 and tuple(pos) in game_state.player_city_tile_xy_set):
            if act[0] == 'build_city':
                if unit.get_cargo_space_used() != 100:
                    continue
                if tuple(unit.pos) not in game_state.buildable_tile_xy_set:
                    continue
                if tuple(unit.pos) in game_state.avoid_building_citytiles_xy_set:
                    print("avoid building", unit.pos, unit.id)
                    continue
            if tuple(pos) in game_state.sinking_cities_xy_set:
                continue
            if tuple(pos) in game_state.opponent_city_tile_xy_set:
                continue
            if unit.fuel_potential == 0 and game_state.turn %40 >= 30:
                if game_state.fuel_collection_rate[pos.y, pos.x] == 0 and tuple(pos) not in game_state.player_city_tile_xy_set:
                    continue
            if unit.fuel_potential > 0 and game_state.matrix_player_cities_nights_of_fuel_required_for_game[pos.y, pos.x] < -20:
                continue
            if act[0] == 'build_city' or unit.pos != pos:
                unit.cooldown += 2
            if act[0] != ('move', 'c'):
                annotations.append(annotate.x(pos.x, pos.y))
            return call_func(unit, *act), pos, annotations

    return unit.move('c'), unit.pos, annotations


def get_imitation_action(observation: Observation, game_state: Game, unit: Unit, DEBUG=False, use_probabilistic_sort=False):
    if DEBUG: print = __builtin__.print
    else: print = lambda *args: None

    start_time = time.time()

    # Worker Actions
    dest = game_state.occupied_xy_set
    state = make_input(observation, unit.id)

    average_policy = np.zeros(6)
    ranked_policy = np.zeros(6)
    NUMBER_OF_TRANSFORMS = game_state.number_of_transforms
    # NUMBER_OF_TRANSFORMS = 1

    with torch.no_grad():

        transformed_states = np.zeros((NUMBER_OF_TRANSFORMS, 20, 32, 32), dtype=np.float32)
        for i, (transform, inv_permute) in enumerate(transforms[:NUMBER_OF_TRANSFORMS]):
            transformed_state = transform(state)
            transformed_states[i,:,:,:] = transformed_state
        transformed_states = torch.from_numpy(transformed_states)

        p = model(transformed_states)
        for (transform, inv_permute), policy in zip(transforms, p.numpy()):
            policy[:4] = policy[inv_permute]
            print(np.round(policy, 2))
            # booster considering transfer actions are discarded
            if tuple(unit.pos) in game_state.wood_exist_xy_set:
                policy[-1] += 0.25

            if game_state.player.researched_coal_projected():
                if tuple(unit.pos) in game_state.coal_exist_xy_set:
                    policy[-1] += 0.75

            if game_state.player.researched_uranium():
                policy[-1] += game_state.convolved_uranium_exist_matrix[unit.pos.y, unit.pos.x]

            if game_state.player.researched_uranium_projected():
                if tuple(unit.pos) in game_state.uranium_exist_xy_set:
                    policy[-1] += 1.25

            average_policy += policy/NUMBER_OF_TRANSFORMS
            ranked_policy += policy.argsort().argsort()

    print(ranked_policy)
    print(average_policy)

    action, pos, annotations = get_action(average_policy, game_state, unit, dest, DEBUG=DEBUG, use_probabilistic_sort=use_probabilistic_sort)
    if tuple(pos):
        dest.add(tuple(pos))
    print(unit.id, unit.pos, pos, action, time.time() - start_time)
    print()

    return [action] + annotations


# Game Rendering
This is a replay of the agent fighting against itself.

The missions of each unit is annotated.
`X` and `O` indicates target position for the unit to move to.
In addition, `O` indicates that the unit will build a citytile upon arrival at the tile.

`O` on the city tile indicates that the citytile have enough fuel to last to the end of the game.
Otherwise, the number of nights it can endure will be indicated on the tile.

The inscription on the unit indicates the amount of total resources it has, and the majority type of resource.
`F` indicates that it has at least 100 resources. If the unit has moved in the turn, the inscription is annotated on the previous location.


In [ ]:
!mkdir snapshots
from kaggle_environments import make
env = make("lux_ai_2021", debug=True, configuration={"annotations": True, "width":12, "height":12})
steps = env.run(["agent.py", "agent.py"])

In [ ]:
env.render(mode="ipython", width=900, height=800)

# Debugging
In the run, we have saved the game state and missions as Python pickle files.

We can rerun the game logic and debug how missions are planned and actions are executed.

For visualisation, we plot `convolved_collectable_tiles_matrix`.
This matrix is used for estimating the best target position of a mission.
You could also print other attributes of `game_state`.


In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from agent import game_logic

str_step = "010"
player_id = 0
with open(f'snapshots/game_state-{str_step}-{player_id}.pkl', 'rb') as handle:
    game_state = pickle.load(handle)
with open(f'snapshots/observation-{str_step}-{player_id}.pkl', 'rb') as handle:
    observation = pickle.load(handle)
with open(f'snapshots/missions-{str_step}-{player_id}.pkl', 'rb') as handle:
    missions = pickle.load(handle)

game_logic(game_state, missions, observation, DEBUG=True)
plt.imshow(game_state.convolved_collectable_tiles_matrix)
plt.colorbar()
plt.show()

# Evaluation
If you want measure the winrate between two agents, you need to play many matches.

For each map size, we play a number of matches. For larger maps, we play a smaller number of matches.

To make scores more comparable, the seed of the matches will have to be consistent over different plays.

In [ ]:
!npm install -g @lux-ai/2021-challenge@latest &> /dev/null
!pip install kaggle-environments -U &> /dev/null

In [ ]:
%%bash
# REF_DIR="/kaggle/input/lux-ai-published-agents/realneuralnetwork/lux-ai-with-il-decreasing-learning-rate/v3/*"
REF_DIR="/kaggle/input/hungry-goose-alphageese-agents/111813_no_curfew/*"
mkdir -p ref/  # imitation agent
cp -r $REF_DIR ref/


In [ ]:
!mkdir template

In [ ]:
%%writefile template/main.py
from typing import Dict
import sys
from agent import agent
if __name__ == "__main__":

    def read_input():
        """
        Reads input from stdin
        """
        try:
            return input()
        except EOFError as eof:
            raise SystemExit(eof)
    step = 0
    class Observation(Dict[str, any]):
        def __init__(self, player=0) -> None:
            self.player = player
            # self.updates = []
            # self.step = 0
    observation = Observation()
    observation["updates"] = []
    observation["step"] = 0
    player_id = 0
    while True:
        inputs = read_input()
        observation["updates"].append(inputs)

        if inputs == "D_DONE":
            if step == 0:  # the codefix
                player_id = int(observation["updates"][0])
                observation.player = player_id
                observation["player"] = player_id
                observation["width"], observation["height"] = map(int, observation["updates"][1].split())
            actions = agent(observation, None)
            observation["updates"] = []
            step += 1
            observation["step"] = step
            print(",".join(actions))
            print("D_FINISH")


In [ ]:
!cd ref/ && tar -xvzf *.tar.gz &> /dev/null
!cp template/main.py ref/main.py  # fix main.py

In [ ]:
!GFOOTBALL_DATA_DIR=C lux-ai-2021 --loglevel 0 --width 12 --height 12 main.py ref/main.py

In [ ]:
%%writefile evaluate_for_map_size.sh

MAP_SIZE=$1
for run in {30000001..30000200};
    do GFOOTBALL_DATA_DIR=C lux-ai-2021 --seed $run --loglevel 1 --maxtime 10000 \
    --height $MAP_SIZE --width $MAP_SIZE --storeReplay=false --storeLogs=false \
    ./main.py ./ref/main.py >> logs-$MAP_SIZE.txt;
done

In [ ]:
!chmod +x ./evaluate_for_map_size.sh

In [ ]:
!timeout 0.5h bash ./evaluate_for_map_size.sh 12

In [ ]:
!timeout 1.5h bash ./evaluate_for_map_size.sh 16

In [ ]:
!timeout 2h bash ./evaluate_for_map_size.sh 24

In [ ]:
!timeout 4h bash ./evaluate_for_map_size.sh 32

In [ ]:
import os

wins_template = """
    { rank: 1, agentID: 0, name: './main.py' },
    { rank: 2, agentID: 1, name: './ref/main.py' }
"""

draw_template = """
    { rank: 1, agentID: 0, name: './main.py' },
    { rank: 1, agentID: 1, name: './ref/main.py' }
"""

lose_template = """
    { rank: 1, agentID: 1, name: './ref/main.py' },
    { rank: 2, agentID: 0, name: './main.py' }
"""

map_sizes = [12,16,24,32]
map_size_count = 0
total_score = 0
for map_size in map_sizes:
    logfile_name = f"logs-{map_size}.txt"
    if os.path.isfile(logfile_name):
        map_size_count += 1
        with open(logfile_name) as f:
            data_string = f.read()
            wins = data_string.count(wins_template)
            draw = data_string.count(draw_template)
            lose = data_string.count(lose_template)
            score = (wins + draw / 2)/(wins + draw + lose)*100
            total_score += score
            print(f"Map size: {map_size}, Score: {score:.3f}, Stats: {wins}/{draw}/{lose}")
total_score = total_score/map_size_count
print(f"Total score: {total_score:.3f}")

In [ ]:
!rm snapshots/*.pkl
!tar --exclude='*.ipynb' --exclude="*.pyc" --exclude="*.pkl" --exclude="./replays/" --exclude="./ref/" -czf submission.tar.gz *
!rm *.py && rm -rf __pycache__/ && rm -rf lux/ && rm -rf ref/